# PO Data Analysis Tool

Interactive analysis of Purchase Order pricing data.

---

## Quick Start (2-3 steps)

### Step 1: Upload Your Data
1. Click the **folder icon** 📁 in the left sidebar to open the Files panel
2. **Drag and drop** your CSV file into the Files panel, OR click the upload button
3. Wait for the upload to complete (you'll see your file appear)

### Step 2: Run the Notebook
- Go to **Runtime** → **Run all** (or press `Ctrl+F9`)

### Step 3: Select Your File *(optional)*
- If your file is named `PO Data.csv`, it will load automatically
- Otherwise, select your file from the dropdown in "Data File Selection" and click **Load Data**

---

## Analysis Sections
- **Section 2**: Monthly Price Trends
- **Section 3**: Weighted Average Calculator
- **Section 4**: Deviation Analysis
- **Section 5**: Deviation Banding
- **Section 6**: Benchmark Comparison

---

## Section 1: Setup & Data Loading

In [ ]:
#@title LIBRARY IMPORTS AND HELPER FUNCTIONS
# =============================================================================
# SECTION 1: LIBRARY IMPORTS AND HELPER FUNCTIONS
# =============================================================================
# pandas: Data manipulation and analysis
# matplotlib: Charts and visualizations
# ipywidgets: Interactive UI widgets (date pickers, dropdowns, buttons)
# =============================================================================

# Try to enable interactive matplotlib (requires ipympl)
# Falls back to inline mode if ipympl is not installed
try:
    %matplotlib widget
    print("Interactive matplotlib enabled - charts have zoom/pan toolbar!")
except:
    %matplotlib inline
    print("Using inline matplotlib (install ipympl for interactive toolbar: pip install ipympl)")

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML, FileLink
import warnings
import os
import base64
from datetime import datetime
warnings.filterwarnings('ignore')

# =============================================================================
# EXPORT HELPER FUNCTIONS
# =============================================================================
# These functions create downloadable CSV files from dataframes
# =============================================================================

def create_export_button(df, filename, button_text="Export to CSV"):
    """
    Create a button that exports a dataframe to CSV when clicked.
    The CSV file is saved and a download link is provided.
    
    Args:
        df: pandas DataFrame to export
        filename: name for the CSV file (without .csv extension)
        button_text: text to display on the button
    
    Returns:
        widgets.VBox containing button and output area
    """
    button = widgets.Button(
        description=button_text,
        button_style='success',
        icon='download'
    )
    output = widgets.Output()
    
    def on_click(b):
        with output:
            output.clear_output()
            # Create exports folder if it doesn't exist
            os.makedirs('exports', exist_ok=True)
            
            # Add timestamp to filename
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            full_filename = f'exports/{filename}_{timestamp}.csv'
            
            # Clean the dataframe - remove HTML tags if present
            clean_df = df.copy()
            for col in clean_df.columns:
                if clean_df[col].dtype == object:
                    clean_df[col] = clean_df[col].astype(str).str.replace(r'<[^>]+>', '', regex=True)
            
            # Save to CSV
            clean_df.to_csv(full_filename, index=True)
            print(f"Exported to: {full_filename}")
            display(FileLink(full_filename))
    
    button.on_click(on_click)
    return widgets.VBox([button, output])

def format_currency(x, decimals=2):
    """Format number as currency with specified decimals"""
    if pd.isna(x):
        return '-'
    return f'${x:,.{decimals}f}'

def format_pct(x, decimals=2):
    """Format number as percentage with specified decimals"""
    if pd.isna(x):
        return '-'
    if x > 0:
        return f'+{x:.{decimals}f}%'
    else:
        return f'{x:.{decimals}f}%'

def format_qty(x):
    """Format number as quantity with commas"""
    if pd.isna(x):
        return '-'
    return f'{int(x):,}'

def color_currency(value, is_gain_loss=True, decimals=2):
    """
    Return HTML-colored string for a currency value.
    For Gain/Loss: positive (savings) = green, negative (loss) = red
    For Price Delta: negative (price down) = green, positive (price up) = red
    """
    if pd.isna(value):
        return '-'
    
    if is_gain_loss:
        # Gain/Loss: positive = savings (green), negative = loss (red)
        if value > 0:
            return f'<span style="color: green; font-weight: bold;">+${value:,.{decimals}f}</span>'
        elif value < 0:
            return f'<span style="color: red; font-weight: bold;">-${abs(value):,.{decimals}f}</span>'
        else:
            return f'$0.{"0" * decimals}'
    else:
        # Price Delta: negative = price went down (green/good), positive = price went up (red/bad)
        if value > 0:
            return f'<span style="color: red;">+${value:.{decimals}f}</span>'
        elif value < 0:
            return f'<span style="color: green;">-${abs(value):.{decimals}f}</span>'
        else:
            return f'$0.{"0" * decimals}'

def color_pct(value, decimals=2):
    """Color percentage: negative = green (price down), positive = red (price up)"""
    if pd.isna(value):
        return '-'
    if value > 0:
        return f'<span style="color: red;">+{value:.{decimals}f}%</span>'
    elif value < 0:
        return f'<span style="color: green;">{value:.{decimals}f}%</span>'
    else:
        return f'0.{"0" * decimals}%'

print("Libraries loaded successfully!")
print("Export functions ready - tables can be exported to CSV files in 'exports' folder")

# =============================================================================
# COLAB OUTPUT EXPANSION  
# =============================================================================
# Remove nested scrollbars by expanding all outputs
# =============================================================================
try:
    from IPython.display import display, HTML
    
    # Aggressive CSS override for Colab's output containers
    display(HTML("""
    <style>
    /* Target all possible Colab output containers */
    .outputarea, .output_area, .output_subarea, .output_scroll,
    .output_wrapper, .output, .cell-output, .cell-output-ipywidget-background,
    [class*="output"], [class*="Output"] {
        max-height: none !important;
        height: auto !important;
        overflow: visible !important;
        overflow-y: visible !important;
        overflow-x: visible !important;
    }
    
    /* Colab specific */
    .colab-df-container, .rendered_html {
        max-height: none !important;
        overflow: visible !important;
    }
    </style>
    
    <script>
    // Expand outputs after they render
    function expandOutputs() {
        document.querySelectorAll('[style*="overflow"]').forEach(el => {
            if (el.style.overflow === 'auto' || el.style.overflowY === 'auto') {
                el.style.overflow = 'visible';
                el.style.overflowY = 'visible';
                el.style.maxHeight = 'none';
                el.style.height = 'auto';
            }
        });
    }
    
    // Run now and observe for new outputs
    expandOutputs();
    setInterval(expandOutputs, 2000);
    </script>
    """))
except:
    pass


In [ ]:
#@title Data File Selection
# =============================================================================
# DATA FILE SELECTION
# =============================================================================
# Select your uploaded CSV file and click Load Data
# =============================================================================

import os
import ipywidgets as widgets
from IPython.display import display, clear_output

# Scan for CSV files (including Colab's /content/ directory)
def find_csv_files():
    csv_files = []
    # Check current directory
    csv_files.extend([f for f in os.listdir('.') if f.endswith('.csv')])
    # Check /content/ for Colab
    if os.path.exists('/content'):
        csv_files.extend([f for f in os.listdir('/content') if f.endswith('.csv') and f not in csv_files])
    return sorted(set(csv_files)) if csv_files else ['(No CSV files found - please upload one)']

csv_files = find_csv_files()

# Create file selector dropdown
file_selector = widgets.Dropdown(
    options=csv_files,
    value=csv_files[0],
    description='Data File:',
    layout=widgets.Layout(width='500px'),
    style={'description_width': '80px'}
)

# Refresh button to rescan for files
refresh_button = widgets.Button(
    description='Refresh List',
    button_style='info',
    icon='refresh',
    layout=widgets.Layout(width='120px')
)

# Create load button
load_button = widgets.Button(
    description='Load Data',
    button_style='primary',
    icon='check'
)

# Output area
load_output = widgets.Output()

# Global variable for loaded data
df_raw = None

def refresh_files(b):
    """Rescan for CSV files."""
    csv_files = find_csv_files()
    file_selector.options = csv_files
    file_selector.value = csv_files[0]
    with load_output:
        clear_output(wait=True)
        print(f"Found {len(csv_files)} CSV file(s)")

def load_data(b):
    """Load data from the selected CSV file."""
    global df_raw
    
    with load_output:
        clear_output(wait=True)
        
        DATA_FILE = file_selector.value
        
        if DATA_FILE.startswith('(No CSV'):
            print("⚠ Please upload a CSV file first!")
            print("  Use the Files panel (folder icon) in the left sidebar.")
            return
        
        print(f"Loading: {DATA_FILE}")
        
        if os.path.exists(DATA_FILE):
            try:
                # Load the data (skiprows=1 for the empty first row)
                df_raw = pd.read_csv(DATA_FILE, skiprows=1)
                print(f"✓ Loaded successfully!")
                print(f"  Rows: {len(df_raw):,}")
                print(f"  Columns: {len(df_raw.columns)}")
                print("\n✓ Ready! Scroll down to use the analysis sections.")
            except Exception as e:
                print(f"✗ Error: {str(e)}")
                df_raw = None
        else:
            print(f"✗ File not found: {DATA_FILE}")
            print("  Click 'Refresh List' after uploading your file.")
            df_raw = None

refresh_button.on_click(refresh_files)
load_button.on_click(load_data)

# Display UI
print("Select your CSV file and click 'Load Data':")
print("(If you just uploaded a file, click 'Refresh List' first)")
print()
display(widgets.HBox([file_selector, refresh_button, load_button]))
display(load_output)

# Auto-load if a real file is selected
if not csv_files[0].startswith('(No CSV'):
    load_data(None)


In [ ]:
#@title DATA CLEANING AND PO-LEVEL AGGREGATION
# =============================================================================
# DATA CLEANING AND PO-LEVEL AGGREGATION
# =============================================================================
# This cell performs critical data cleaning steps. PAY ATTENTION to the
# Purchase Price parsing - there's a known issue with the dollar sign.
#
# IMPORTANT: We aggregate rows at the PO# + Material level because multiple
# rows with the same PO# just represent different SIZES of the same order.
# The price is the same across sizes, so we sum the quantities.
# =============================================================================

df = df_raw.copy()

# -----------------------------------------------------------------------------
# PURCHASE PRICE PARSING - CRITICAL!
# -----------------------------------------------------------------------------
# The ' Purchase Price ' column (note: has spaces in name!) contains values
# formatted as ' $41.70 ' (with leading/trailing spaces AND a dollar sign).
#
# WHAT DOESN'T WORK:
#   df['Price'] = df[' Purchase Price '].str.replace('$', '', regex=False).astype(float)
#   This fails because regex=False doesn't handle '$' consistently in pandas.
#
# SOLUTION: Use regex=True with escaped dollar sign '\\$'
# The '$' has special meaning in regex (end of string), so we escape it.
# -----------------------------------------------------------------------------
df['Purchase_Price'] = pd.to_numeric(
    df[' Purchase Price '].astype(str).str.replace('\\$', '', regex=True).str.strip(),
    errors='coerce'  # Convert unparseable values to NaN instead of raising error
)

# -----------------------------------------------------------------------------
# PO DATE PARSING
# -----------------------------------------------------------------------------
# Format: M/DD/YY (e.g., '1/29/25' = January 29, 2025)
# IMPORTANT: Always use 'PO Date' for time-based analysis!
# Do NOT use other date fields (XFD, ETA, Delivery dates, etc.)
# -----------------------------------------------------------------------------
df['PO_Date'] = pd.to_datetime(df['PO Date'], format='%m/%d/%y', errors='coerce')

# -----------------------------------------------------------------------------
# ORDERED QUANTITY
# -----------------------------------------------------------------------------
# Column name has apostrophe: "Ordered Q'ty"
# Convert to numeric and fill NaN with 0
# -----------------------------------------------------------------------------
df['Ordered_Qty'] = pd.to_numeric(df["Ordered Q'ty"], errors='coerce').fillna(0).astype(int)

# -----------------------------------------------------------------------------
# FILTER OUT ZERO-QUANTITY ROWS
# -----------------------------------------------------------------------------
# Rows with Ordered Qty = 0 don't contribute to weighted averages
# and would cause division issues, so we exclude them
# -----------------------------------------------------------------------------
df = df[df['Ordered_Qty'] > 0].copy()

print(f"After initial cleaning: {len(df)} rows (before PO aggregation)")

# -----------------------------------------------------------------------------
# PO-LEVEL AGGREGATION - COMBINE SIZE VARIATIONS
# -----------------------------------------------------------------------------
# Multiple rows with the same PO# + Material represent different SIZES of the
# same order. Since sizes are irrelevant for price analysis (same price per unit),
# we aggregate by summing quantities while keeping the price.
#
# Category columns preserved for filtering (from CSV):
#   - Material Category Description
#   - Material Line Description
#   - Material Product Type Description
#   - Material Class Description
#   - Construction Type Description
# -----------------------------------------------------------------------------
df = df.groupby(['PO#', 'Material', 'Short Name', 'PO_Date', 'Purchase_Price']).agg({
    'Ordered_Qty': 'sum',                              # Sum quantities across all sizes
    'Vendor Name': 'first',                            # Keep vendor info
    'Color': 'first',                                  # Keep color info
    'Material Category Description': 'first',          # Category for filtering
    'Material Line Description': 'first',              # Line for filtering
    'Material Product Type Description': 'first',      # Type for filtering
    'Material Class Description': 'first',             # Class for filtering
    'Construction Type Description': 'first'           # Construction type for filtering
}).reset_index()

print(f"After PO aggregation: {len(df)} rows (combined sizes within same PO)")

# -----------------------------------------------------------------------------
# ADD HELPER COLUMNS
# -----------------------------------------------------------------------------
# PO_Month: For grouping by month (format: '2025-01')
# Total_Value: Price * Qty for weighted average calculations
# Product_Label: Human-readable label for charts (Material ID + Short Name)
# -----------------------------------------------------------------------------
df['PO_Month'] = df['PO_Date'].dt.to_period('M').astype(str)
df['Total_Value'] = df['Purchase_Price'] * df['Ordered_Qty']
df['Product_Label'] = df['Material'].astype(str) + ' - ' + df['Short Name'].fillna('')

# -----------------------------------------------------------------------------
# CREATE PRODUCT-LEVEL LOOKUP FOR FILTERING
# -----------------------------------------------------------------------------
# This maps each Material to its category attributes (for filter dropdowns)
# Using the exact columns from the CSV file
# -----------------------------------------------------------------------------
product_categories = df.groupby('Material').agg({
    'Short Name': 'first',
    'Product_Label': 'first',
    'Material Category Description': 'first',
    'Material Line Description': 'first',
    'Material Product Type Description': 'first',
    'Material Class Description': 'first',
    'Construction Type Description': 'first'
}).reset_index()

# -----------------------------------------------------------------------------
# CREATE CATEGORY OPTIONS FOR FILTER DROPDOWNS
# -----------------------------------------------------------------------------
# These lists are used by all sections (3, 4, 5, 6) for category-based filtering
# Each list contains unique non-null values sorted alphabetically
# Format: (display_label, value) tuples for use with widgets.Dropdown
# -----------------------------------------------------------------------------
category_options = sorted([v for v in df['Material Category Description'].unique() if pd.notna(v)])
line_options = sorted([v for v in df['Material Line Description'].unique() if pd.notna(v)])
type_options = sorted([v for v in df['Material Product Type Description'].unique() if pd.notna(v)])
class_options = sorted([v for v in df['Material Class Description'].unique() if pd.notna(v)])
construction_options = sorted([v for v in df['Construction Type Description'].unique() if pd.notna(v)])

def get_materials_by_filters(category=None, line=None, mat_type=None, mat_class=None, construction=None):
    """
    Get list of Material IDs that match ALL specified filter criteria.
    If a filter is None or empty, it's ignored (not applied).
    Returns list of Material IDs.
    """
    mask = pd.Series([True] * len(product_categories))
    
    if category and category != 'All':
        mask &= (product_categories['Material Category Description'] == category)
    if line and line != 'All':
        mask &= (product_categories['Material Line Description'] == line)
    if mat_type and mat_type != 'All':
        mask &= (product_categories['Material Product Type Description'] == mat_type)
    if mat_class and mat_class != 'All':
        mask &= (product_categories['Material Class Description'] == mat_class)
    if construction and construction != 'All':
        mask &= (product_categories['Construction Type Description'] == construction)
    
    return product_categories[mask]['Material'].tolist()


# =============================================================================
# SCROLLABLE TABLE HELPER FUNCTION
# =============================================================================
# This function wraps a DataFrame in a scrollable HTML container
# Used when "Show All Rows" toggle is enabled
# =============================================================================

def display_scrollable_table(df, max_height='400px', show_all=False):
    """
    Display a DataFrame, optionally showing all rows.
    
    Parameters:
    - df: pandas DataFrame to display
    - max_height: Ignored (kept for backwards compatibility)
    - show_all: If True, shows all rows. If False, uses default pandas display.
    
    Returns: HTML widget or DataFrame for display
    """
    if show_all:
        # Convert to HTML with all rows (no scrollable container)
        html_table = df.to_html(index=True, escape=False)
        return widgets.HTML(value=html_table)
    else:
        # Return the dataframe for standard display (pandas will truncate)
        return df

def create_show_all_toggle(description='Show All Rows'):
    """Create a toggle checkbox for showing all rows."""
    return widgets.Checkbox(
        value=False,
        description=description,
        indent=False,
        layout=widgets.Layout(width='150px')
    )

print(f"\nDate range: {df['PO_Date'].min().strftime('%Y-%m-%d')} to {df['PO_Date'].max().strftime('%Y-%m-%d')}")
print(f"Unique POs: {df['PO#'].nunique()}")
print(f"Unique products (Materials): {df['Material'].nunique()}")

# Show category breakdowns for filtering
print(f"\nFilter categories available:")
print(f"  Material Category: {len(category_options)} options")
print(f"  Material Line: {len(line_options)} options")
print(f"  Material Type: {len(type_options)} options")
print(f"  Material Class: {len(class_options)} options")
print(f"  Construction Type: {len(construction_options)} options")

print("\nSample data (note: Ordered_Qty is now aggregated across sizes):")
df[['PO#', 'Material', 'Short Name', 'PO_Date', 'PO_Month', 'Ordered_Qty', 'Purchase_Price', 'Total_Value']].head(10)

## Section 2: Monthly Price Trend Visualization

This section shows the weighted average price trend for each product over time, grouped by PO Date month.

In [ ]:
#@title calculate average price when order sizes vary.
# =============================================================================
# CALCULATE MONTHLY WEIGHTED AVERAGE PRICE PER PRODUCT
# =============================================================================
# Formula: Weighted Avg = SUM(Price * Qty) / SUM(Qty)
#
# This gives more weight to larger orders, which is the correct way to 
# calculate average price when order sizes vary.
#
# Example: If you buy 10 units at $50 and 100 units at $45:
#   Simple average: ($50 + $45) / 2 = $47.50
#   Weighted average: (10*$50 + 100*$45) / 110 = $45.45  <-- This is correct!
# =============================================================================

def calculate_monthly_trends(data):
    """
    Calculate weighted average price per Material per month.
    
    Groups by Material ID (not Short Name) to aggregate all sizes/colors.
    Returns dataframe with one row per product-month combination.
    """
    monthly = data.groupby(['Material', 'PO_Month']).agg({
        'Total_Value': 'sum',      # Sum of (Price * Qty) for weighted avg numerator
        'Ordered_Qty': 'sum',      # Sum of Qty for weighted avg denominator
        'Short Name': 'first',     # Keep product name for display
        'Product_Label': 'first'   # Keep label for chart legends
    }).reset_index()
    
    # Weighted Average = Total Value / Total Quantity
    monthly['Weighted_Avg_Price'] = monthly['Total_Value'] / monthly['Ordered_Qty']
    monthly = monthly.sort_values(['Material', 'PO_Month'])
    
    return monthly

monthly_trends = calculate_monthly_trends(df)
print(f"Monthly trend data points: {len(monthly_trends)}")
monthly_trends.head(10)

In [ ]:
#@title INTERACTIVE MONTHLY TREND CHART
# =============================================================================
# INTERACTIVE MONTHLY TREND CHART
# =============================================================================
# This creates a multi-select widget to choose which products to display
# on the trend chart. Hold Ctrl/Cmd to select multiple products.
#
# BY DEFAULT: All products are shown. User can deselect to focus on specific ones.
#
# The chart uses matplotlib with interactive toolbar (zoom, pan, save).
# Each product gets a distinct color.
#
# CATEGORY FILTERS: Filter products by Category, Line, Type, Class, Construction
# =============================================================================

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

# Build dropdown options: (display_label, material_id) pairs
products = df[['Material', 'Product_Label']].drop_duplicates().sort_values('Product_Label')
product_options = [(row['Product_Label'], row['Material']) for _, row in products.iterrows()]

# Get all material IDs for default selection (show ALL products initially)
all_material_ids = [mat_id for _, mat_id in product_options]

# =============================================================================
# CATEGORY FILTER DROPDOWNS
# =============================================================================
trend_category_dropdown = widgets.Dropdown(
    options=['All'] + category_options,
    value='All',
    description='Category:',
    layout=widgets.Layout(width='400px')
)

trend_line_dropdown = widgets.Dropdown(
    options=['All'] + line_options,
    value='All',
    description='Line:',
    layout=widgets.Layout(width='400px')
)

trend_type_dropdown = widgets.Dropdown(
    options=['All'] + type_options,
    value='All',
    description='Type:',
    layout=widgets.Layout(width='400px')
)

trend_class_dropdown = widgets.Dropdown(
    options=['All'] + class_options,
    value='All',
    description='Class:',
    layout=widgets.Layout(width='400px')
)

trend_construction_dropdown = widgets.Dropdown(
    options=['All'] + construction_options,
    value='All',
    description='Construction:',
    layout=widgets.Layout(width='400px')
)

trend_filter_status = widgets.HTML(value='<b>Matching products:</b> All')

# Multi-select widget for product selection - DEFAULT TO ALL SELECTED
product_selector = widgets.SelectMultiple(
    options=product_options,
    value=all_material_ids,  # Default: ALL products selected
    description='Products:',
    disabled=False,
    layout=widgets.Layout(width='600px', height='250px')
)

def update_trend_product_list(*args):
    """Update product selector based on category filter selections."""
    matching_materials = get_materials_by_filters(
        category=trend_category_dropdown.value,
        line=trend_line_dropdown.value,
        mat_type=trend_type_dropdown.value,
        mat_class=trend_class_dropdown.value,
        construction=trend_construction_dropdown.value
    )
    
    # Filter product options to only matching materials
    filtered_options = [(label, mat_id) for label, mat_id in product_options if mat_id in matching_materials]
    
    # Update product selector with filtered options
    product_selector.options = filtered_options
    
    # Select all matching products by default
    product_selector.value = [mat_id for _, mat_id in filtered_options]
    
    # Update filter status
    if len(filtered_options) == len(product_options):
        trend_filter_status.value = '<b>Matching products:</b> All'
    else:
        trend_filter_status.value = f'<b>Matching products:</b> {len(filtered_options)} of {len(product_options)}'

# Connect category filter observers
trend_category_dropdown.observe(update_trend_product_list, names='value')
trend_line_dropdown.observe(update_trend_product_list, names='value')
trend_type_dropdown.observe(update_trend_product_list, names='value')
trend_class_dropdown.observe(update_trend_product_list, names='value')
trend_construction_dropdown.observe(update_trend_product_list, names='value')

# Buttons for quick selection
select_all_btn = widgets.Button(description='Select All', button_style='info')
clear_all_btn = widgets.Button(description='Clear All', button_style='warning')

def select_all(b):
    # Select all currently visible (filtered) products
    product_selector.value = [mat_id for _, mat_id in product_selector.options]
    
def clear_all(b):
    product_selector.value = []

select_all_btn.on_click(select_all)
clear_all_btn.on_click(clear_all)

# Output area for the chart
trend_output = widgets.Output()

# Store figure reference for cleanup
trend_fig = None

def update_trend_chart(change):
    """
    Callback function triggered when product selection changes.
    Filters monthly_trends data and renders a matplotlib line chart.
    Each product gets a distinct color from a colormap.
    
    INTERACTIVE TOOLBAR: Use the toolbar below the chart to:
    - Pan: Click and drag to move around
    - Zoom: Draw a rectangle to zoom in, right-click to zoom out
    - Home: Reset to original view
    - Save: Save the chart as an image
    """
    global trend_fig
    
    with trend_output:
        trend_output.clear_output(wait=True)
        
        # Close previous figure if exists to prevent memory leak
        if trend_fig is not None:
            plt.close(trend_fig)
        
        selected_materials = list(product_selector.value)
        
        if not selected_materials:
            print("Please select at least one product (or click 'Select All').")
            return
        
        # Filter to selected products only
        filtered = monthly_trends[monthly_trends['Material'].isin(selected_materials)]
        
        # Get unique products
        unique_products = filtered['Product_Label'].unique()
        num_products = len(unique_products)
        
        # Create color map for products
        if num_products <= 10:
            colors = plt.cm.tab10(np.linspace(0, 1, 10))[:num_products]
        elif num_products <= 20:
            colors = plt.cm.tab20(np.linspace(0, 1, 20))[:num_products]
        else:
            colors = plt.cm.viridis(np.linspace(0, 1, num_products))
        
        # Create figure - dynamic height based on number of products
        # Extra height for legend below the chart
        fig_height = max(10, 8 + num_products * 0.08)  # Minimum 10 inches to fit legend below
        fig_width = 12
        
        trend_fig, ax = plt.subplots(figsize=(fig_width, fig_height))
        
        # Plot each product
        for i, product in enumerate(unique_products):
            product_data = filtered[filtered['Product_Label'] == product].sort_values('PO_Month')
            ax.plot(product_data['PO_Month'], product_data['Weighted_Avg_Price'], 
                   marker='o', markersize=5, linewidth=1.5, color=colors[i], label=product)
        
        ax.set_xlabel('Month', fontsize=11)
        ax.set_ylabel('Weighted Average Price ($)', fontsize=11)
        ax.set_title(f'Monthly Weighted Average Price Trend ({num_products} products)', fontsize=13, fontweight='bold')
        
        # Rotate x-axis labels for readability
        plt.xticks(rotation=45, ha='right')
        
        # Add gridlines
        ax.yaxis.grid(True, linestyle='--', alpha=0.7)
        ax.set_axisbelow(True)
        
        # Legend - place below the chart
        # Use multiple columns to fit more items horizontally
        num_cols = min(4, num_products)  # Max 4 columns, fewer if less products
        ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), fontsize=8, 
                  frameon=True, fancybox=True, shadow=True, ncol=num_cols)
        
        # Adjust layout to make room for legend below
        plt.subplots_adjust(bottom=0.25)
        plt.show()
        
        # Print summary and toolbar instructions
        print(f"\nShowing {num_products} products across {filtered['PO_Month'].nunique()} months")
        print("\nToolbar: Use buttons below chart to Pan, Zoom, Reset view, or Save image")

# Connect the callback to the widget
product_selector.observe(update_trend_chart, names='value')

# =============================================================================
# DISPLAY THE INTERFACE
# =============================================================================
print("Filter products by category, then select specific products to view trends:")
print("(Hold Ctrl/Cmd to select multiple, or use buttons below)")
print()

# Category filters
print("Filter by Category:")
display(widgets.VBox([
    widgets.HBox([trend_category_dropdown, trend_line_dropdown]),
    widgets.HBox([trend_type_dropdown, trend_class_dropdown]),
    trend_construction_dropdown,
    trend_filter_status
]))

print()
print("Select Products:")
display(widgets.HBox([select_all_btn, clear_all_btn]))
display(product_selector)
display(trend_output)

# Trigger initial chart render with ALL products
update_trend_chart(None)


In [ ]:
#@title MONTHLY PRICE TREND DATA TABLE WITH EXPORT
# =============================================================================
# MONTHLY PRICE TREND DATA TABLE WITH EXPORT
# =============================================================================
# This table shows the raw data behind the chart above:
# - Each row = one Product + Month combination
# - Weighted Avg Price = SUM(Price * Qty) / SUM(Qty) for that month
# - Total Qty = total units ordered that month
# All numbers formatted to 2 decimal places
# =============================================================================

# Show All Rows toggle for Section 2 tables
section2_show_all_rows = widgets.Checkbox(
    value=False,
    description='Show All Rows (scrollable)',
    indent=False,
    layout=widgets.Layout(width='250px')
)

# Output area for tables
section2_table_output = widgets.Output()

def display_section2_tables(change=None):
    """Display the Section 2 tables with optional scrolling."""
    with section2_table_output:
        section2_table_output.clear_output(wait=True)
        
        show_all = section2_show_all_rows.value
        
        # Create a pivot table view: Products as rows, Months as columns
        pivot_price = monthly_trends.pivot_table(
            index=['Material', 'Short Name'],
            columns='PO_Month',
            values='Weighted_Avg_Price',
            aggfunc='first'
        ).round(2)

        # Store raw data for export
        pivot_price_raw = pivot_price.copy()

        # Format prices with $ sign (2 decimals)
        pivot_price_display = pivot_price.applymap(lambda x: f'${x:.2f}' if pd.notna(x) else '-')

        print("=== Monthly Weighted Average Price by Product ===")
        print(f"(Rows: {len(pivot_price)} products, Columns: {len(pivot_price.columns)} months)\n")
        
        if show_all:
            display(display_scrollable_table(pivot_price_display, max_height='400px', show_all=True))
        else:
            display(pivot_price_display)

        # Export button for pivot table
        display(create_export_button(pivot_price_raw, 'monthly_price_trend_pivot', 'Export Price Pivot to CSV'))

        # Also show a detailed table sorted by product and month
        print("\n\n=== Detailed Monthly Trend Data ===")
        detail_table = monthly_trends[['Material', 'Short Name', 'PO_Month', 'Weighted_Avg_Price', 'Ordered_Qty']].copy()
        detail_table_raw = detail_table.copy()  # For export
        detail_table = detail_table.sort_values(['Short Name', 'PO_Month'])
        detail_table['Weighted_Avg_Price'] = detail_table['Weighted_Avg_Price'].apply(lambda x: f'${x:.2f}')
        detail_table['Ordered_Qty'] = detail_table['Ordered_Qty'].apply(lambda x: f'{x:,}')
        detail_table = detail_table.rename(columns={
            'Short Name': 'Product',
            'PO_Month': 'Month',
            'Weighted_Avg_Price': 'Wtd Avg Price',
            'Ordered_Qty': 'Total Qty'
        })
        
        if show_all:
            display(display_scrollable_table(detail_table, max_height='500px', show_all=True))
        else:
            display(detail_table)

        # Export button for detailed table
        detail_table_raw = detail_table_raw.sort_values(['Short Name', 'PO_Month'])
        detail_table_raw = detail_table_raw.rename(columns={
            'Short Name': 'Product',
            'PO_Month': 'Month',
            'Weighted_Avg_Price': 'Wtd_Avg_Price',
            'Ordered_Qty': 'Total_Qty'
        })
        display(create_export_button(detail_table_raw, 'monthly_price_trend_detail', 'Export Detail Table to CSV'))

# Connect toggle to refresh tables
section2_show_all_rows.observe(display_section2_tables, names='value')

# Display toggle and tables
print("DISPLAY OPTIONS:")
display(section2_show_all_rows)
print()
display_section2_tables()
display(section2_table_output)


## Section 3: Weighted Average Price Calculator

Calculate weighted average price for a selected date range, optionally filtered by product.

In [ ]:
#@title 1. Select a date range (start and end dates)
# =============================================================================
# WEIGHTED AVERAGE CALCULATOR - INTERACTIVE WITH EXPORT
# =============================================================================
# This section allows you to:
# 1. Select a date range (start and end dates)
# 2. Filter by product categories (Material Category, Line, Type, Class, Construction)
# 3. Optionally filter to specific individual products
# 4. Calculate weighted average price for each product in that range
#
# Output includes:
# - Summary table with Weighted Avg, Total Quantity, Total FOB Value
# - Excel-format table showing monthly price trends within the selected range
# - Export buttons for each table
#
# All numbers formatted to 2 decimal places
# Toggle "Show All Rows" to view all data in scrollable tables
# =============================================================================

# Get date boundaries from the data
min_date = df['PO_Date'].min().date()
max_date = df['PO_Date'].max().date()

# Date picker widgets
start_date_picker = widgets.DatePicker(
    description='Start Date:',
    value=min_date,
    disabled=False
)

end_date_picker = widgets.DatePicker(
    description='End Date:',
    value=max_date,
    disabled=False
)

# -----------------------------------------------------------------------------
# CATEGORY FILTER DROPDOWNS
# -----------------------------------------------------------------------------
# These dropdowns filter products by category BEFORE showing in the product list
# User can select from any category dimension to narrow down products
# -----------------------------------------------------------------------------

category_dropdown = widgets.Dropdown(
    options=['All'] + category_options,
    value='All',
    description='Category:',
    layout=widgets.Layout(width='400px')
)

line_dropdown = widgets.Dropdown(
    options=['All'] + line_options,
    value='All',
    description='Line:',
    layout=widgets.Layout(width='400px')
)

type_dropdown = widgets.Dropdown(
    options=['All'] + type_options,
    value='All',
    description='Type:',
    layout=widgets.Layout(width='400px')
)

class_dropdown = widgets.Dropdown(
    options=['All'] + class_options,
    value='All',
    description='Class:',
    layout=widgets.Layout(width='400px')
)

construction_dropdown = widgets.Dropdown(
    options=['All'] + construction_options,
    value='All',
    description='Construction:',
    layout=widgets.Layout(width='400px')
)

# Product filter - will be updated based on category selections
product_filter = widgets.SelectMultiple(
    options=[('All Products', 'ALL')] + product_options,
    value=['ALL'],  # Default to all products
    description='Products:',
    layout=widgets.Layout(width='600px', height='150px')
)

# Label showing how many products match current filters
filter_status = widgets.HTML(value='<b>Matching products:</b> All')

# Show All Rows toggle
show_all_rows_toggle = widgets.Checkbox(
    value=False,
    description='Show All Rows (scrollable)',
    indent=False,
    layout=widgets.Layout(width='200px')
)

def update_product_list(*args):
    """Update the product list based on category filter selections."""
    # Get materials matching all selected category filters
    matching_materials = get_materials_by_filters(
        category=category_dropdown.value,
        line=line_dropdown.value,
        mat_type=type_dropdown.value,
        mat_class=class_dropdown.value,
        construction=construction_dropdown.value
    )
    
    # Filter product_options to only show matching products
    filtered_options = [(label, mat_id) for label, mat_id in product_options if mat_id in matching_materials]
    
    # Update the product selector
    product_filter.options = [('All Matching Products', 'ALL')] + filtered_options
    product_filter.value = ['ALL']
    
    # Update status label
    filter_status.value = f'<b>Matching products:</b> {len(filtered_options)} of {len(product_options)}'

# Connect category dropdowns to update function
category_dropdown.observe(update_product_list, names='value')
line_dropdown.observe(update_product_list, names='value')
type_dropdown.observe(update_product_list, names='value')
class_dropdown.observe(update_product_list, names='value')
construction_dropdown.observe(update_product_list, names='value')

# Button to trigger calculation
calc_button = widgets.Button(description='Calculate', button_style='primary')
calc_output = widgets.Output()

# Store dataframes for export (will be populated when calculation runs)
section3_exports = {}

def calculate_weighted_avg(b):
    """
    Calculate and display weighted average prices for selected criteria.
    
    Outputs:
    1. Summary table with overall weighted averages per product
    2. Excel-format table showing monthly trends within the selected date range
    3. Export buttons for each table
    """
    global section3_exports
    
    with calc_output:
        calc_output.clear_output(wait=True)
        
        # Check if Show All Rows is enabled
        show_all = show_all_rows_toggle.value
        
        # Convert widget dates to pandas Timestamps for comparison
        start = pd.Timestamp(start_date_picker.value)
        end = pd.Timestamp(end_date_picker.value)
        
        # Filter by date range
        filtered = df[(df['PO_Date'] >= start) & (df['PO_Date'] <= end)].copy()
        
        # Get materials matching category filters
        matching_materials = get_materials_by_filters(
            category=category_dropdown.value,
            line=line_dropdown.value,
            mat_type=type_dropdown.value,
            mat_class=class_dropdown.value,
            construction=construction_dropdown.value
        )
        
        # Apply category filter
        filtered = filtered[filtered['Material'].isin(matching_materials)]
        
        # Filter by specific products if selected (not 'ALL')
        selected_products = list(product_filter.value)
        if 'ALL' not in selected_products:
            filtered = filtered[filtered['Material'].isin(selected_products)]
        
        if len(filtered) == 0:
            print("No data found for the selected criteria.")
            return
        
        # =================================================================
        # HEADER - Show selected date range and filters prominently
        # =================================================================
        print("=" * 80)
        print(f"WEIGHTED AVERAGE PRICE ANALYSIS")
        print("=" * 80)
        print(f"Selected Date Range: {start.strftime('%Y-%m-%d')} to {end.strftime('%Y-%m-%d')}")
        
        # Show active category filters
        active_filters = []
        if category_dropdown.value != 'All':
            active_filters.append(f"Category: {category_dropdown.value}")
        if line_dropdown.value != 'All':
            active_filters.append(f"Line: {line_dropdown.value}")
        if type_dropdown.value != 'All':
            active_filters.append(f"Type: {type_dropdown.value}")
        if class_dropdown.value != 'All':
            active_filters.append(f"Class: {class_dropdown.value}")
        if construction_dropdown.value != 'All':
            active_filters.append(f"Construction: {construction_dropdown.value}")
        
        if active_filters:
            print(f"Category Filters: {', '.join(active_filters)}")
        else:
            print("Category Filters: None (all categories)")
            
        print(f"Total PO Line Items: {len(filtered)}")
        print(f"Unique Products: {filtered['Material'].nunique()}")
        if show_all:
            print("Display Mode: Showing ALL rows (scrollable)")
        print("=" * 80)
        
        # =================================================================
        # TABLE 1: Overall Weighted Average Summary
        # =================================================================
        summary = filtered.groupby(['Material', 'Short Name']).agg({
            'Total_Value': 'sum',   # Numerator for weighted avg
            'Ordered_Qty': 'sum'    # Denominator for weighted avg
        }).reset_index()
        
        summary['Weighted_Avg_Price'] = (summary['Total_Value'] / summary['Ordered_Qty']).round(2)
        summary['Total_Value'] = summary['Total_Value'].round(2)
        summary = summary.rename(columns={
            'Material': 'Material_ID',
            'Short Name': 'Product_Name',
            'Ordered_Qty': 'Total_Quantity',
            'Total_Value': 'Total_FOB_Value',
            'Weighted_Avg_Price': 'Weighted_Avg_Price'
        })
        
        summary = summary.sort_values('Product_Name')
        
        # Store for export
        section3_exports['summary'] = summary.copy()
        
        print(f"\n--- OVERALL WEIGHTED AVERAGES (for entire date range) ---\n")
        
        # Format numbers for display (2 decimals)
        display_df = summary.copy()
        display_df['Weighted_Avg_Price'] = display_df['Weighted_Avg_Price'].apply(lambda x: f'${x:.2f}')
        display_df['Total_FOB_Value'] = display_df['Total_FOB_Value'].apply(lambda x: f'${x:,.2f}')
        display_df['Total_Quantity'] = display_df['Total_Quantity'].apply(lambda x: f'{x:,}')
        display_df.columns = ['Material ID', 'Product Name', 'Total Quantity', 'Total FOB Value ($)', 'Weighted Avg Price ($)']
        
        # Display with scrollable option if Show All Rows is enabled
        if show_all:
            display(display_scrollable_table(display_df, max_height='400px', show_all=True))
        else:
            display(display_df)
        display(create_export_button(section3_exports['summary'], 'section3_weighted_avg_summary', 'Export Summary to CSV'))
        
        # =================================================================
        # TABLE 2: Monthly Trend (Excel-format pivot table)
        # =================================================================
        print(f"\n\n--- MONTHLY PRICE TREND (within selected date range) ---")
        print(f"Date Range: {start.strftime('%Y-%m-%d')} to {end.strftime('%Y-%m-%d')}\n")
        
        # Calculate monthly weighted averages within the filtered data
        monthly = filtered.groupby(['Material', 'Short Name', 'PO_Month']).agg({
            'Total_Value': 'sum',
            'Ordered_Qty': 'sum'
        }).reset_index()
        monthly['Monthly_Avg'] = (monthly['Total_Value'] / monthly['Ordered_Qty']).round(2)
        
        # Create pivot table: rows = products, columns = months
        pivot_price = monthly.pivot_table(
            index=['Material', 'Short Name'],
            columns='PO_Month',
            values='Monthly_Avg',
            aggfunc='first'
        ).round(2)
        
        # Store for export
        section3_exports['monthly_price'] = pivot_price.copy()
        
        # Get the overall weighted avg for each product (for the last column)
        summary_indexed = summary.set_index(['Material_ID', 'Product_Name'])
        
        # Format prices with $ sign for display (2 decimals)
        pivot_display = pivot_price.copy()
        for col in pivot_display.columns:
            pivot_display[col] = pivot_display[col].apply(lambda x: f'${x:.2f}' if pd.notna(x) else '-')
        
        # Add overall weighted average as the last column
        pivot_display['Overall Wtd Avg'] = summary_indexed['Weighted_Avg_Price'].apply(lambda x: f'${x:.2f}')
        
        print("Excel-Format Table: Monthly Weighted Average Price by Product")
        print("(Each cell = weighted avg price for that product in that month)")
        print("")
        
        # Display with scrollable option if Show All Rows is enabled
        if show_all:
            display(display_scrollable_table(pivot_display, max_height='400px', show_all=True))
        else:
            display(pivot_display)
        display(create_export_button(section3_exports['monthly_price'], 'section3_monthly_price_pivot', 'Export Monthly Prices to CSV'))
        
        # =================================================================
        # TABLE 3: Monthly Quantity Trend
        # =================================================================
        print(f"\n\n--- MONTHLY QUANTITY TREND (within selected date range) ---")
        print(f"Date Range: {start.strftime('%Y-%m-%d')} to {end.strftime('%Y-%m-%d')}\n")
        
        pivot_qty = monthly.pivot_table(
            index=['Material', 'Short Name'],
            columns='PO_Month',
            values='Ordered_Qty',
            aggfunc='sum'
        ).fillna(0).astype(int)
        
        # Store for export
        section3_exports['monthly_qty'] = pivot_qty.copy()
        
        # Format quantities
        pivot_qty_display = pivot_qty.copy()
        for col in pivot_qty_display.columns:
            pivot_qty_display[col] = pivot_qty_display[col].apply(lambda x: f'{x:,}' if x > 0 else '-')
        
        # Add total qty column
        pivot_qty_display['Total Qty'] = summary_indexed['Total_Quantity'].apply(lambda x: f'{x:,}')
        
        print("Excel-Format Table: Monthly Quantity Ordered by Product")
        print("(Each cell = total quantity ordered for that product in that month)")
        print("")
        
        # Display with scrollable option if Show All Rows is enabled
        if show_all:
            display(display_scrollable_table(pivot_qty_display, max_height='400px', show_all=True))
        else:
            display(pivot_qty_display)
        display(create_export_button(section3_exports['monthly_qty'], 'section3_monthly_qty_pivot', 'Export Monthly Quantities to CSV'))

# Connect button click to callback
calc_button.on_click(calculate_weighted_avg)

# Display the interface
print("=" * 80)
print("WEIGHTED AVERAGE CALCULATOR")
print("=" * 80)
print("\n1. SELECT DATE RANGE:")
display(widgets.HBox([start_date_picker, end_date_picker]))

print("\n2. FILTER BY CATEGORY (optional - narrow down products):")
print("   Select from any category to filter the product list below.")
display(widgets.VBox([
    widgets.HBox([category_dropdown, line_dropdown]),
    widgets.HBox([type_dropdown, class_dropdown]),
    construction_dropdown
]))

print("\n3. SELECT PRODUCTS:")
display(filter_status)
print("   Select 'All Matching Products' or choose specific products (Ctrl+click for multiple):")
display(product_filter)

print("\n4. DISPLAY OPTIONS:")
display(show_all_rows_toggle)

print("\n5. CALCULATE:")
display(calc_button)
display(calc_output)


## Section 4: Deviation Analysis

Calculate how individual line items deviate from the weighted average price.

In [ ]:
#@title DEVIATION ANALYSIS - INTERACTIVE WITH EXPORT
# =============================================================================
# DEVIATION ANALYSIS - INTERACTIVE WITH EXPORT
# =============================================================================
# This section shows how individual PO line items deviate from the weighted
# average price for their product.
#
# For each PO line item, we calculate:
#   Dollar Deviation = Item Price - Weighted Average
#   Percent Deviation = (Dollar Deviation / Weighted Average) * 100
#
# COLOR CODING:
#   GREEN = Below average (SAVINGS - you paid less!)
#   RED = Above average (LOSS - you paid more!)
#
# OUTPUT TABLES:
# 1. Monthly Deviation Summary (Excel format) - Shows each product's monthly
#    weighted avg price deviation from the overall weighted avg
# 2. Individual PO Line Item Deviations - Detailed list of each PO
# 3. Export buttons for all tables
#
# All numbers formatted to 2 decimal places
# Toggle "Show All Rows" to view all data in scrollable tables
# =============================================================================

# Date pickers (separate from Section 3 to allow different ranges)
dev_start_picker = widgets.DatePicker(
    description='Start Date:',
    value=min_date,
    disabled=False
)

dev_end_picker = widgets.DatePicker(
    description='End Date:',
    value=max_date,
    disabled=False
)

# -----------------------------------------------------------------------------
# CATEGORY FILTER DROPDOWNS FOR SECTION 4
# -----------------------------------------------------------------------------
dev_category_dropdown = widgets.Dropdown(
    options=['All'] + category_options,
    value='All',
    description='Category:',
    layout=widgets.Layout(width='400px')
)

dev_line_dropdown = widgets.Dropdown(
    options=['All'] + line_options,
    value='All',
    description='Line:',
    layout=widgets.Layout(width='400px')
)

dev_type_dropdown = widgets.Dropdown(
    options=['All'] + type_options,
    value='All',
    description='Type:',
    layout=widgets.Layout(width='400px')
)

dev_class_dropdown = widgets.Dropdown(
    options=['All'] + class_options,
    value='All',
    description='Class:',
    layout=widgets.Layout(width='400px')
)

dev_construction_dropdown = widgets.Dropdown(
    options=['All'] + construction_options,
    value='All',
    description='Construction:',
    layout=widgets.Layout(width='400px')
)

# Product filter
dev_product_filter = widgets.SelectMultiple(
    options=[('All Products', 'ALL')] + product_options,
    value=['ALL'],
    description='Products:',
    layout=widgets.Layout(width='600px', height='150px')
)

# Label showing how many products match current filters
dev_filter_status = widgets.HTML(value='<b>Matching products:</b> All')

# Show All Rows toggle for Section 4
dev_show_all_rows_toggle = widgets.Checkbox(
    value=False,
    description='Show All Rows (scrollable)',
    indent=False,
    layout=widgets.Layout(width='200px')
)

def update_dev_product_list(*args):
    """Update the product list based on category filter selections."""
    matching_materials = get_materials_by_filters(
        category=dev_category_dropdown.value,
        line=dev_line_dropdown.value,
        mat_type=dev_type_dropdown.value,
        mat_class=dev_class_dropdown.value,
        construction=dev_construction_dropdown.value
    )
    
    filtered_options = [(label, mat_id) for label, mat_id in product_options if mat_id in matching_materials]
    dev_product_filter.options = [('All Matching Products', 'ALL')] + filtered_options
    dev_product_filter.value = ['ALL']
    dev_filter_status.value = f'<b>Matching products:</b> {len(filtered_options)} of {len(product_options)}'

# Connect category dropdowns to update function
dev_category_dropdown.observe(update_dev_product_list, names='value')
dev_line_dropdown.observe(update_dev_product_list, names='value')
dev_type_dropdown.observe(update_dev_product_list, names='value')
dev_class_dropdown.observe(update_dev_product_list, names='value')
dev_construction_dropdown.observe(update_dev_product_list, names='value')

# Threshold filter dropdown
threshold_type = widgets.Dropdown(
    options=[
        ('All Items', 'all'),           # Show everything
        ('Above Average', 'above'),     # Items with positive deviation
        ('Below Average', 'below')      # Items with negative deviation
    ],
    value='all',
    description='Filter:'
)

# Threshold amount (only applies when above/below selected)
threshold_value = widgets.FloatText(
    value=0.0,
    description='Threshold ($):',
    disabled=False
)

dev_button = widgets.Button(description='Analyze Deviations', button_style='primary')
dev_output = widgets.Output()

# Store dataframes for export
section4_exports = {}

def color_deviation(value, is_dollar=True, decimals=2):
    """
    Color code deviation values for display.
    
    For DEVIATIONS (price difference from average):
      NEGATIVE deviation = GOOD (paid less than average = GREEN)
      POSITIVE deviation = BAD (paid more than average = RED)
    
    This is OPPOSITE of gain/loss coloring!
    """
    if pd.isna(value):
        return '-'
    
    if is_dollar:
        if value < 0:
            # Negative deviation = savings = GREEN
            return f'<span style="color: green; font-weight: bold;">-${abs(value):.{decimals}f}</span>'
        elif value > 0:
            # Positive deviation = loss = RED
            return f'<span style="color: red; font-weight: bold;">+${value:.{decimals}f}</span>'
        else:
            return f'$0.00'
    else:
        # Percentage
        if value < 0:
            return f'<span style="color: green; font-weight: bold;">{value:.{decimals}f}%</span>'
        elif value > 0:
            return f'<span style="color: red; font-weight: bold;">+{value:.{decimals}f}%</span>'
        else:
            return f'0.00%'

def analyze_deviations(b):
    """
    Calculate deviation for each PO line item from its product's weighted average.
    
    Output:
    1. Monthly Deviation Summary Table (Excel format)
    2. Individual PO Line Items with deviations
    3. Export buttons for all tables
    """
    global section4_exports
    
    with dev_output:
        dev_output.clear_output(wait=True)
        
        # Check if Show All Rows is enabled
        show_all = dev_show_all_rows_toggle.value
        
        start = pd.Timestamp(dev_start_picker.value)
        end = pd.Timestamp(dev_end_picker.value)
        
        # Filter by date
        filtered = df[(df['PO_Date'] >= start) & (df['PO_Date'] <= end)].copy()
        
        # Get materials matching category filters
        matching_materials = get_materials_by_filters(
            category=dev_category_dropdown.value,
            line=dev_line_dropdown.value,
            mat_type=dev_type_dropdown.value,
            mat_class=dev_class_dropdown.value,
            construction=dev_construction_dropdown.value
        )
        
        # Apply category filter
        filtered = filtered[filtered['Material'].isin(matching_materials)]
        
        # Filter by specific products if not 'ALL'
        selected_products = list(dev_product_filter.value)
        if 'ALL' not in selected_products:
            filtered = filtered[filtered['Material'].isin(selected_products)]
        
        if len(filtered) == 0:
            print("No data found for the selected criteria.")
            return
        
        # =================================================================
        # HEADER
        # =================================================================
        print("=" * 80)
        print("DEVIATION ANALYSIS")
        print("=" * 80)
        print(f"Selected Date Range: {start.strftime('%Y-%m-%d')} to {end.strftime('%Y-%m-%d')}")
        
        # Show active category filters
        active_filters = []
        if dev_category_dropdown.value != 'All':
            active_filters.append(f"Category: {dev_category_dropdown.value}")
        if dev_line_dropdown.value != 'All':
            active_filters.append(f"Line: {dev_line_dropdown.value}")
        if dev_type_dropdown.value != 'All':
            active_filters.append(f"Type: {dev_type_dropdown.value}")
        if dev_class_dropdown.value != 'All':
            active_filters.append(f"Class: {dev_class_dropdown.value}")
        if dev_construction_dropdown.value != 'All':
            active_filters.append(f"Construction: {dev_construction_dropdown.value}")
        
        if active_filters:
            print(f"Category Filters: {', '.join(active_filters)}")
        else:
            print("Category Filters: None (all categories)")
            
        print(f"Total PO Line Items: {len(filtered)}")
        print(f"Unique Products: {filtered['Material'].nunique()}")
        if show_all:
            print("Display Mode: Showing ALL rows (scrollable)")
        print("")
        print("COLOR KEY: GREEN = Below Average (Savings) | RED = Above Average (Paid More)")
        print("=" * 80)
        
        # =================================================================
        # Calculate OVERALL weighted average per product (for the entire date range)
        # This is the baseline to compare monthly averages against
        # =================================================================
        product_avgs = filtered.groupby(['Material', 'Short Name']).agg({
            'Total_Value': 'sum',
            'Ordered_Qty': 'sum'
        }).reset_index()
        product_avgs['Overall_Weighted_Avg'] = (product_avgs['Total_Value'] / product_avgs['Ordered_Qty']).round(2)
        
        # =================================================================
        # TABLE 1: MONTHLY DEVIATION SUMMARY (Excel Format)
        # Shows how each month's weighted avg deviates from overall weighted avg
        # =================================================================
        print("\n--- MONTHLY PRICE DEVIATION FROM OVERALL WEIGHTED AVERAGE ---")
        print(f"Date Range: {start.strftime('%Y-%m-%d')} to {end.strftime('%Y-%m-%d')}")
        print("GREEN = month's avg was BELOW overall avg (savings)")
        print("RED = month's avg was ABOVE overall avg (paid more)\n")
        
        # Calculate monthly weighted averages
        monthly = filtered.groupby(['Material', 'Short Name', 'PO_Month']).agg({
            'Total_Value': 'sum',
            'Ordered_Qty': 'sum'
        }).reset_index()
        monthly['Monthly_Avg'] = (monthly['Total_Value'] / monthly['Ordered_Qty']).round(2)
        
        # Merge with overall averages
        monthly = monthly.merge(
            product_avgs[['Material', 'Overall_Weighted_Avg']], 
            on='Material'
        )
        
        # Calculate deviation: Monthly Avg - Overall Avg
        monthly['Monthly_Deviation'] = (monthly['Monthly_Avg'] - monthly['Overall_Weighted_Avg']).round(2)
        monthly['Monthly_Deviation_Pct'] = ((monthly['Monthly_Deviation'] / monthly['Overall_Weighted_Avg']) * 100).round(2)
        
        # Create pivot table for DOLLAR deviation
        pivot_dev = monthly.pivot_table(
            index=['Material', 'Short Name'],
            columns='PO_Month',
            values='Monthly_Deviation',
            aggfunc='first'
        ).round(2)
        
        # Store for export
        section4_exports['monthly_deviation_dollars'] = pivot_dev.copy()
        
        # Add overall weighted avg as last column for reference
        overall_avg_series = product_avgs.set_index(['Material', 'Short Name'])['Overall_Weighted_Avg'].round(2)
        
        # Format the deviation values with color coding
        pivot_dev_display = pivot_dev.copy()
        for col in pivot_dev_display.columns:
            pivot_dev_display[col] = pivot_dev_display[col].apply(lambda x: color_deviation(x, is_dollar=True, decimals=2))
        
        # Add overall weighted avg column
        pivot_dev_display['Overall Wtd Avg'] = overall_avg_series.apply(lambda x: f'${x:.2f}')
        
        print("Table 1: Monthly Deviation in DOLLARS from Overall Weighted Average")
        print("(Last column shows the overall weighted avg for reference)\n")
        
        # Display with scrollable option if Show All Rows is enabled
        if show_all:
            html_table = pivot_dev_display.to_html(escape=False)
            scrollable_html = f'<div style="max-height: 400px; overflow-y: auto; overflow-x: auto; border: 1px solid #ddd; padding: 5px;">{html_table}</div>'
            display(HTML(scrollable_html))
        else:
            display(HTML(pivot_dev_display.to_html(escape=False)))
        display(create_export_button(section4_exports['monthly_deviation_dollars'], 'section4_monthly_deviation_dollars', 'Export Dollar Deviations to CSV'))
        
        # Create pivot table for PERCENTAGE deviation
        pivot_pct = monthly.pivot_table(
            index=['Material', 'Short Name'],
            columns='PO_Month',
            values='Monthly_Deviation_Pct',
            aggfunc='first'
        ).round(2)
        
        # Store for export
        section4_exports['monthly_deviation_pct'] = pivot_pct.copy()
        
        # Format percentage values with color coding
        pivot_pct_display = pivot_pct.copy()
        for col in pivot_pct_display.columns:
            pivot_pct_display[col] = pivot_pct_display[col].apply(lambda x: color_deviation(x, is_dollar=False, decimals=2))
        
        print("\n\nTable 2: Monthly Deviation in PERCENTAGE from Overall Weighted Average\n")
        
        # Display with scrollable option if Show All Rows is enabled
        if show_all:
            html_table = pivot_pct_display.to_html(escape=False)
            scrollable_html = f'<div style="max-height: 400px; overflow-y: auto; overflow-x: auto; border: 1px solid #ddd; padding: 5px;">{html_table}</div>'
            display(HTML(scrollable_html))
        else:
            display(HTML(pivot_pct_display.to_html(escape=False)))
        display(create_export_button(section4_exports['monthly_deviation_pct'], 'section4_monthly_deviation_pct', 'Export Percentage Deviations to CSV'))
        
        # =================================================================
        # TABLE 3: Monthly Weighted Average Prices (for context)
        # =================================================================
        pivot_price = monthly.pivot_table(
            index=['Material', 'Short Name'],
            columns='PO_Month',
            values='Monthly_Avg',
            aggfunc='first'
        ).round(2)
        
        # Store for export
        section4_exports['monthly_prices'] = pivot_price.copy()
        
        pivot_price_display = pivot_price.copy()
        for col in pivot_price_display.columns:
            pivot_price_display[col] = pivot_price_display[col].apply(
                lambda x: f'${x:.2f}' if pd.notna(x) else '-'
            )
        pivot_price_display['Overall Wtd Avg'] = overall_avg_series.apply(lambda x: f'${x:.2f}')
        
        print("\n\nTable 3: Monthly Weighted Average Prices (for reference)\n")
        
        # Display with scrollable option if Show All Rows is enabled
        if show_all:
            display(display_scrollable_table(pivot_price_display, max_height='400px', show_all=True))
        else:
            display(pivot_price_display)
        display(create_export_button(section4_exports['monthly_prices'], 'section4_monthly_prices', 'Export Monthly Prices to CSV'))
        
        # =================================================================
        # TABLE 4: INDIVIDUAL PO LINE ITEM DEVIATIONS
        # =================================================================
        print("\n" + "=" * 80)
        print("INDIVIDUAL PO LINE ITEM DEVIATIONS")
        print("=" * 80)
        
        # Merge weighted averages back to each line item
        result = filtered.merge(product_avgs[['Material', 'Overall_Weighted_Avg']], on='Material')
        
        # Calculate deviations for individual items (2 decimals)
        result['Dollar_Deviation'] = (result['Purchase_Price'] - result['Overall_Weighted_Avg']).round(2)
        result['Pct_Deviation'] = ((result['Dollar_Deviation'] / result['Overall_Weighted_Avg']) * 100).round(2)
        
        # Apply threshold filter
        if threshold_type.value == 'above':
            result = result[result['Dollar_Deviation'] >= threshold_value.value]
        elif threshold_type.value == 'below':
            result = result[result['Dollar_Deviation'] <= -threshold_value.value]
        
        # Prepare dataframe for export (raw numbers)
        export_cols = ['PO#', 'Material', 'Short Name', 'PO_Date', 'Purchase_Price', 'Overall_Weighted_Avg', 'Dollar_Deviation', 'Pct_Deviation', 'Ordered_Qty']
        result_export = result[export_cols].copy()
        result_export['PO_Date'] = result_export['PO_Date'].dt.strftime('%Y-%m-%d')
        result_export = result_export.sort_values('Dollar_Deviation', ascending=False)
        section4_exports['individual_deviations'] = result_export.copy()
        
        # Prepare display dataframe
        result_display = result[export_cols].copy()
        result_display = result_display.rename(columns={
            'Short Name': 'Product',
            'PO_Date': 'PO Date',
            'Purchase_Price': 'Unit Price',
            'Overall_Weighted_Avg': 'Wtd Avg',
            'Dollar_Deviation': 'Dev ($)',
            'Pct_Deviation': 'Dev (%)',
            'Ordered_Qty': 'Qty'
        })
        
        # Sort by deviation (largest positive first - shows worst cases first)
        result_display = result_display.sort_values('Dev ($)', ascending=False)
        
        # Display results header
        print(f"Filter: {threshold_type.value}")
        if threshold_type.value != 'all':
            print(f"Threshold: ${threshold_value.value:.2f}")
        print(f"Results: {len(result_display)} PO line items")
        print(f"Total Quantity: {result['Ordered_Qty'].sum():,} units\n")
        
        # Format for display with color coding (2 decimals)
        result_display_html = result_display.copy()
        result_display_html['PO Date'] = result_display_html['PO Date'].dt.strftime('%Y-%m-%d')
        result_display_html['Unit Price'] = result_display_html['Unit Price'].apply(lambda x: f'${x:.2f}')
        result_display_html['Wtd Avg'] = result_display_html['Wtd Avg'].apply(lambda x: f'${x:.2f}')
        result_display_html['Dev ($)'] = result_display_html['Dev ($)'].apply(lambda x: color_deviation(x, is_dollar=True, decimals=2))
        result_display_html['Dev (%)'] = result_display_html['Dev (%)'].apply(lambda x: color_deviation(x, is_dollar=False, decimals=2))
        result_display_html['Qty'] = result_display_html['Qty'].apply(lambda x: f'{x:,}')
        
        # Display with scrollable option if Show All Rows is enabled
        if show_all:
            html_table = result_display_html.to_html(escape=False, index=False)
            scrollable_html = f'<div style="max-height: 500px; overflow-y: auto; overflow-x: auto; border: 1px solid #ddd; padding: 5px;">{html_table}</div>'
            display(HTML(scrollable_html))
            print(f"\nShowing all {len(result_display)} results (scrollable)")
        else:
            # Show first 100 rows (for performance)
            display(HTML(result_display_html.head(100).to_html(escape=False, index=False)))
            if len(result_display) > 100:
                print(f"\n... showing first 100 of {len(result_display)} results")
                print("Enable 'Show All Rows' to see all results in scrollable table")
        
        display(create_export_button(section4_exports['individual_deviations'], 'section4_individual_deviations', 'Export All Individual Deviations to CSV'))

dev_button.on_click(analyze_deviations)

# Display the interface
print("=" * 80)
print("DEVIATION ANALYSIS")
print("=" * 80)
print("\n1. SELECT DATE RANGE:")
display(widgets.HBox([dev_start_picker, dev_end_picker]))

print("\n2. FILTER BY CATEGORY (optional - narrow down products):")
display(widgets.VBox([
    widgets.HBox([dev_category_dropdown, dev_line_dropdown]),
    widgets.HBox([dev_type_dropdown, dev_class_dropdown]),
    dev_construction_dropdown
]))

print("\n3. SELECT PRODUCTS:")
display(dev_filter_status)
display(dev_product_filter)

print("\n4. DEVIATION THRESHOLD FILTER (for individual PO table):")
display(widgets.HBox([threshold_type, threshold_value]))

print("\n5. DISPLAY OPTIONS:")
display(dev_show_all_rows_toggle)

print("\n6. ANALYZE:")
display(dev_button)
display(dev_output)


## Section 5: Deviation Banding

Categorize items into $1 deviation bands and visualize the distribution.

In [ ]:
#@title DEVIATION BANDING - HISTOGRAM AND PIE CHART
# =============================================================================
# DEVIATION BANDING - HISTOGRAM AND PIE CHART
# =============================================================================
# This section categorizes PO line items into $1 deviation bands and visualizes
# the distribution using:
#   1. Histogram: TOTAL QUANTITY in each band (not count of items!)
#   2. Pie Chart: Percentage distribution by quantity across bands
#
# WHY TOTAL QUANTITY? Because what matters is "HOW MUCH YOU BOUGHT AT THAT PRICE"
# A single PO for 10,000 units at a bad price is worse than 10 POs for 100 units.
#
# Bands range from "$5+ below" to "$5+ above" in $1 increments.
#
# COLORS:
#   GREEN = Below average (savings - you paid less than average!)
#   RED = Above average (loss - you paid more than average!)
#
# INTERACTIVE TOOLBAR: Charts have zoom/pan/save buttons
# Toggle "Show All Rows" to view all data in scrollable tables
# =============================================================================

import matplotlib.pyplot as plt

# Date pickers
band_start_picker = widgets.DatePicker(
    description='Start Date:',
    value=min_date,
    disabled=False
)

band_end_picker = widgets.DatePicker(
    description='End Date:',
    value=max_date,
    disabled=False
)

# -----------------------------------------------------------------------------
# CATEGORY FILTER DROPDOWNS FOR SECTION 5
# -----------------------------------------------------------------------------
band_category_dropdown = widgets.Dropdown(
    options=['All'] + category_options,
    value='All',
    description='Category:',
    layout=widgets.Layout(width='400px')
)

band_line_dropdown = widgets.Dropdown(
    options=['All'] + line_options,
    value='All',
    description='Line:',
    layout=widgets.Layout(width='400px')
)

band_type_dropdown = widgets.Dropdown(
    options=['All'] + type_options,
    value='All',
    description='Type:',
    layout=widgets.Layout(width='400px')
)

band_class_dropdown = widgets.Dropdown(
    options=['All'] + class_options,
    value='All',
    description='Class:',
    layout=widgets.Layout(width='400px')
)

band_construction_dropdown = widgets.Dropdown(
    options=['All'] + construction_options,
    value='All',
    description='Construction:',
    layout=widgets.Layout(width='400px')
)

# Product filter
band_product_filter = widgets.SelectMultiple(
    options=[('All Products', 'ALL')] + product_options,
    value=['ALL'],
    description='Products:',
    layout=widgets.Layout(width='600px', height='150px')
)

# Label showing how many products match current filters
band_filter_status = widgets.HTML(value='<b>Matching products:</b> All')

# Show All Rows toggle for Section 5
band_show_all_rows_toggle = widgets.Checkbox(
    value=False,
    description='Show All Rows (scrollable)',
    indent=False,
    layout=widgets.Layout(width='200px')
)

def update_band_product_list(*args):
    """Update the product list based on category filter selections."""
    matching_materials = get_materials_by_filters(
        category=band_category_dropdown.value,
        line=band_line_dropdown.value,
        mat_type=band_type_dropdown.value,
        mat_class=band_class_dropdown.value,
        construction=band_construction_dropdown.value
    )
    
    filtered_options = [(label, mat_id) for label, mat_id in product_options if mat_id in matching_materials]
    band_product_filter.options = [('All Matching Products', 'ALL')] + filtered_options
    band_product_filter.value = ['ALL']
    band_filter_status.value = f'<b>Matching products:</b> {len(filtered_options)} of {len(product_options)}'

# Connect category dropdowns to update function
band_category_dropdown.observe(update_band_product_list, names='value')
band_line_dropdown.observe(update_band_product_list, names='value')
band_type_dropdown.observe(update_band_product_list, names='value')
band_class_dropdown.observe(update_band_product_list, names='value')
band_construction_dropdown.observe(update_band_product_list, names='value')

band_button = widgets.Button(description='Generate Banding Charts', button_style='primary')
band_output = widgets.Output()

# Store figure references for cleanup
band_fig_hist = None
band_fig_pie = None

def assign_band(deviation):
    """
    Assign a dollar deviation value to a named band.
    
    Bands are $1 wide, from "$5+ below" to "$5+ above".
    """
    if deviation <= -5:
        return '$5+ below'
    elif deviation <= -4:
        return '$4-5 below'
    elif deviation <= -3:
        return '$3-4 below'
    elif deviation <= -2:
        return '$2-3 below'
    elif deviation <= -1:
        return '$1-2 below'
    elif deviation < 0:
        return '$0-1 below'
    elif deviation < 1:
        return '$0-1 above'
    elif deviation < 2:
        return '$1-2 above'
    elif deviation < 3:
        return '$2-3 above'
    elif deviation < 4:
        return '$3-4 above'
    elif deviation < 5:
        return '$4-5 above'
    else:
        return '$5+ above'

# Define band order for consistent chart display (worst to best)
band_order = [
    '$5+ below', '$4-5 below', '$3-4 below', '$2-3 below',
    '$1-2 below', '$0-1 below', '$0-1 above', '$1-2 above',
    '$2-3 above', '$3-4 above', '$4-5 above', '$5+ above'
]

def generate_banding(b):
    """
    Generate histogram and pie chart showing deviation band distribution.
    
    IMPORTANT: Uses TOTAL QUANTITY per band, not count of line items!
    Uses matplotlib with interactive toolbar.
    """
    global band_fig_hist, band_fig_pie
    
    with band_output:
        band_output.clear_output(wait=True)
        
        # Close previous figures if they exist
        if band_fig_hist is not None:
            plt.close(band_fig_hist)
        if band_fig_pie is not None:
            plt.close(band_fig_pie)
        
        # Check if Show All Rows is enabled
        show_all = band_show_all_rows_toggle.value
        
        start = pd.Timestamp(band_start_picker.value)
        end = pd.Timestamp(band_end_picker.value)
        
        # Filter by date
        filtered = df[(df['PO_Date'] >= start) & (df['PO_Date'] <= end)].copy()
        
        # Get materials matching category filters
        matching_materials = get_materials_by_filters(
            category=band_category_dropdown.value,
            line=band_line_dropdown.value,
            mat_type=band_type_dropdown.value,
            mat_class=band_class_dropdown.value,
            construction=band_construction_dropdown.value
        )
        
        # Apply category filter
        filtered = filtered[filtered['Material'].isin(matching_materials)]
        
        # Filter by specific products if not 'ALL'
        selected_products = list(band_product_filter.value)
        if 'ALL' not in selected_products:
            filtered = filtered[filtered['Material'].isin(selected_products)]
        
        if len(filtered) == 0:
            print("No data found for the selected criteria.")
            return
        
        # Calculate weighted average per product
        product_avgs = filtered.groupby('Material').agg({
            'Total_Value': 'sum',
            'Ordered_Qty': 'sum'
        }).reset_index()
        product_avgs['Weighted_Avg'] = product_avgs['Total_Value'] / product_avgs['Ordered_Qty']
        
        # Calculate deviations
        result = filtered.merge(product_avgs[['Material', 'Weighted_Avg']], on='Material')
        result['Dollar_Deviation'] = result['Purchase_Price'] - result['Weighted_Avg']
        
        # Assign each item to a band
        result['Band'] = result['Dollar_Deviation'].apply(assign_band)
        
        # SUM TOTAL QUANTITY per band (not count!)
        band_qty = result.groupby('Band')['Ordered_Qty'].sum().reindex(band_order, fill_value=0).reset_index()
        band_qty.columns = ['Band', 'Total_Qty']
        
        # Also get count for reference
        band_counts = result['Band'].value_counts().reindex(band_order, fill_value=0).reset_index()
        band_counts.columns = ['Band', 'PO_Count']
        
        # Merge for display
        band_summary = band_qty.merge(band_counts, on='Band')
        
        print("=" * 80)
        print("DEVIATION BANDING ANALYSIS")
        print("=" * 80)
        print(f"Date Range: {start.strftime('%Y-%m-%d')} to {end.strftime('%Y-%m-%d')}")
        
        # Show active category filters
        active_filters = []
        if band_category_dropdown.value != 'All':
            active_filters.append(f"Category: {band_category_dropdown.value}")
        if band_line_dropdown.value != 'All':
            active_filters.append(f"Line: {band_line_dropdown.value}")
        if band_type_dropdown.value != 'All':
            active_filters.append(f"Type: {band_type_dropdown.value}")
        if band_class_dropdown.value != 'All':
            active_filters.append(f"Class: {band_class_dropdown.value}")
        if band_construction_dropdown.value != 'All':
            active_filters.append(f"Construction: {band_construction_dropdown.value}")
        
        if active_filters:
            print(f"Category Filters: {', '.join(active_filters)}")
        else:
            print("Category Filters: None (all categories)")
            
        print(f"Total PO Line Items: {len(result)}")
        print(f"Total Quantity: {result['Ordered_Qty'].sum():,} units")
        if show_all:
            print("Display Mode: Showing ALL rows (scrollable)")
        print("=" * 80)
        print("\nToolbar: Use buttons below each chart to Pan, Zoom, Reset view, or Save image")
        
        # ---------------------------------------------------------------------
        # HISTOGRAM - USING MATPLOTLIB WITH INTERACTIVE TOOLBAR
        # ---------------------------------------------------------------------
        # GREEN for "below average" bands (savings)
        # RED for "above average" bands (paid more)
        colors = ['#2ca02c'] * 6 + ['#d62728'] * 6
        
        band_fig_hist, ax = plt.subplots(figsize=(12, 5))
        
        x_pos = range(len(band_order))
        quantities = band_summary['Total_Qty'].tolist()
        
        bars = ax.bar(x_pos, quantities, color=colors, edgecolor='black', linewidth=0.5)
        
        # Add value labels on top of bars
        for bar, qty in zip(bars, quantities):
            if qty > 0:
                ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(quantities)*0.01, 
                        f'{qty:,}', ha='center', va='bottom', fontsize=8, fontweight='bold')
        
        ax.set_xticks(x_pos)
        ax.set_xticklabels(band_order, rotation=45, ha='right', fontsize=9)
        ax.set_xlabel('Deviation Band', fontsize=11)
        ax.set_ylabel('Total Quantity (units)', fontsize=11)
        ax.set_title('Deviation Banding by TOTAL QUANTITY Purchased\n(Green = Below Average / Savings | Red = Above Average / Paid More)', 
                     fontsize=12, fontweight='bold')
        
        # Add gridlines for readability
        ax.yaxis.grid(True, linestyle='--', alpha=0.7)
        ax.set_axisbelow(True)
        
        plt.tight_layout()
        plt.show()
        
        # ---------------------------------------------------------------------
        # PIE CHART - USING LEGEND BELOW TO AVOID LABEL OVERLAP
        # ---------------------------------------------------------------------
        # Small slices cause label overlap, so we use a legend instead
        # Percentages shown inside large slices, legend explains all colors
        # ---------------------------------------------------------------------
        pie_colors = {
            '$5+ below': '#2ca02c', '$4-5 below': '#3cb371', '$3-4 below': '#32cd32',
            '$2-3 below': '#90ee90', '$1-2 below': '#98fb98', '$0-1 below': '#b2f2bb',
            '$0-1 above': '#ffb3b3', '$1-2 above': '#ff8080', '$2-3 above': '#ff6666',
            '$3-4 above': '#ff4d4d', '$4-5 above': '#e62e2e', '$5+ above': '#d62728'
        }
        
        # Filter out bands with zero quantity for pie chart
        pie_data = band_summary[band_summary['Total_Qty'] > 0]
        
        if len(pie_data) > 0:
            band_fig_pie, ax2 = plt.subplots(figsize=(10, 10))
            
            pie_labels = pie_data['Band'].tolist()
            pie_values = pie_data['Total_Qty'].tolist()
            pie_colors_list = [pie_colors.get(b, '#888888') for b in pie_labels]
            
            # Calculate percentages for display
            total = sum(pie_values)
            percentages = [(v / total) * 100 for v in pie_values]
            
            # Create slight explosion for visual separation
            explode = [0.03] * len(pie_values)
            
            # Create pie chart with percentages inside (no external labels)
            def make_autopct(values):
                def my_autopct(pct):
                    if pct >= 3:  # Only show percentage if slice is >= 3%
                        return f'{pct:.1f}%'
                    return ''
                return my_autopct
            
            wedges, texts, autotexts = ax2.pie(
                pie_values, 
                labels=None,  # No labels on pie - use legend instead
                colors=pie_colors_list,
                autopct=make_autopct(pie_values),
                startangle=90,
                explode=explode,
                pctdistance=0.6,
                wedgeprops={
                    'edgecolor': 'white',
                    'linewidth': 2.5,
                    'antialiased': True
                },
                shadow=True
            )
            
            # Style the percentage text
            for autotext in autotexts:
                autotext.set_fontsize(11)
                autotext.set_fontweight('bold')
                autotext.set_color('black')
            
            # Create legend labels with band name and percentage
            legend_labels = [f'{label} ({pct:.1f}%)' for label, pct in zip(pie_labels, percentages)]
            
            # Add legend below the pie chart
            ax2.legend(wedges, legend_labels, 
                      title="Deviation Bands",
                      loc='upper center', 
                      bbox_to_anchor=(0.5, -0.05),
                      ncol=3,
                      fontsize=9,
                      title_fontsize=10,
                      frameon=True,
                      fancybox=True,
                      shadow=True)
            
            ax2.set_title('Quantity Distribution by Deviation Band\n(Green = Savings | Red = Paid More)', 
                         fontsize=14, fontweight='bold')
            
            # Equal aspect ratio ensures pie is circular
            ax2.axis('equal')
            
            plt.tight_layout()
            plt.subplots_adjust(bottom=0.2)  # Make room for legend
            plt.show()
        
        # Summary table with both count and quantity
        print("\nBand Summary:")
        band_summary_display = band_summary.copy()
        band_summary_display['Qty %'] = (band_summary_display['Total_Qty'] / band_summary_display['Total_Qty'].sum() * 100).round(1).astype(str) + '%'
        band_summary_display['Total_Qty'] = band_summary_display['Total_Qty'].apply(lambda x: f'{x:,}')
        
        # Display with scrollable option if Show All Rows is enabled
        if show_all:
            display(display_scrollable_table(band_summary_display, max_height='400px', show_all=True))
        else:
            display(band_summary_display)
        
        # Additional stats
        below_qty = result[result['Dollar_Deviation'] < 0]['Ordered_Qty'].sum()
        above_qty = result[result['Dollar_Deviation'] >= 0]['Ordered_Qty'].sum()
        total_qty = result['Ordered_Qty'].sum()
        print(f"\nSummary:")
        print(f"  Quantity bought BELOW average: {below_qty:,} ({below_qty/total_qty*100:.1f}%) - GREEN = SAVINGS")
        print(f"  Quantity bought AT/ABOVE average: {above_qty:,} ({above_qty/total_qty*100:.1f}%) - RED = PAID MORE")

band_button.on_click(generate_banding)

# Display the interface
print("=" * 80)
print("DEVIATION BANDING")
print("=" * 80)
print("Categorize purchases into deviation bands by TOTAL QUANTITY:")
print("(Shows HOW MUCH you bought at each price deviation level)")
print("COLOR KEY: Green = Below Avg (Savings) | Red = Above Avg (Paid More)")

print("\n1. SELECT DATE RANGE:")
display(widgets.HBox([band_start_picker, band_end_picker]))

print("\n2. FILTER BY CATEGORY (optional - narrow down products):")
display(widgets.VBox([
    widgets.HBox([band_category_dropdown, band_line_dropdown]),
    widgets.HBox([band_type_dropdown, band_class_dropdown]),
    band_construction_dropdown
]))

print("\n3. SELECT PRODUCTS:")
display(band_filter_status)
display(band_product_filter)

print("\n4. DISPLAY OPTIONS:")
display(band_show_all_rows_toggle)

print("\n5. GENERATE CHARTS:")
display(band_button)
display(band_output)


## Section 6: Benchmark Comparison

Compare pricing between two different date ranges to identify price changes.

In [ ]:
#@title HOW IT WORKS:
# =============================================================================
# BENCHMARK COMPARISON - PER-PRODUCT ANALYSIS WITH EXPORT
# =============================================================================
# This section compares pricing between two time periods using PER-PRODUCT
# baseline prices (fair comparison - no mixing of different price products).
#
# HOW IT WORKS:
# 1. BASELINE Period: Calculate weighted average price FOR EACH PRODUCT
# 2. COMPARISON Period: For each product, compare against ITS OWN baseline
# 3. Gain/Loss = (Baseline Price - Comparison Price) × Comparison Qty
#    - Calculated PER PRODUCT, then summed for total
#
# This ensures fair comparison:
#   - $20 product compared to its $20 baseline
#   - $50 product compared to its $50 baseline
#   - No mixing of apples and oranges!
#
# COLOR CODING:
#    GREEN = Savings (you paid less than baseline)
#    RED = Loss (you paid more than baseline)
#
# All numbers formatted to 2 decimal places
# Toggle "Show All Rows" to view all data in scrollable tables
# =============================================================================

# -----------------------------------------------------------------------------
# CATEGORY FILTER DROPDOWNS FOR SECTION 6
# -----------------------------------------------------------------------------
bench_category_dropdown = widgets.Dropdown(
    options=['All'] + category_options,
    value='All',
    description='Category:',
    layout=widgets.Layout(width='400px')
)

bench_line_dropdown = widgets.Dropdown(
    options=['All'] + line_options,
    value='All',
    description='Line:',
    layout=widgets.Layout(width='400px')
)

bench_type_dropdown = widgets.Dropdown(
    options=['All'] + type_options,
    value='All',
    description='Type:',
    layout=widgets.Layout(width='400px')
)

bench_class_dropdown = widgets.Dropdown(
    options=['All'] + class_options,
    value='All',
    description='Class:',
    layout=widgets.Layout(width='400px')
)

bench_construction_dropdown = widgets.Dropdown(
    options=['All'] + construction_options,
    value='All',
    description='Construction:',
    layout=widgets.Layout(width='400px')
)

# Product filter
benchmark_product_filter = widgets.SelectMultiple(
    options=[('All Products', 'ALL')] + product_options,
    value=['ALL'],
    description='Products:',
    layout=widgets.Layout(width='600px', height='150px')
)

# Label showing how many products match current filters
bench_filter_status = widgets.HTML(value='<b>Matching products:</b> All')

# Show All Rows toggle for Section 6
bench_show_all_rows_toggle = widgets.Checkbox(
    value=False,
    description='Show All Rows (scrollable)',
    indent=False,
    layout=widgets.Layout(width='200px')
)

def update_bench_product_list(*args):
    """Update the product list based on category filter selections."""
    matching_materials = get_materials_by_filters(
        category=bench_category_dropdown.value,
        line=bench_line_dropdown.value,
        mat_type=bench_type_dropdown.value,
        mat_class=bench_class_dropdown.value,
        construction=bench_construction_dropdown.value
    )
    
    filtered_options = [(label, mat_id) for label, mat_id in product_options if mat_id in matching_materials]
    benchmark_product_filter.options = [('All Matching Products', 'ALL')] + filtered_options
    benchmark_product_filter.value = ['ALL']
    bench_filter_status.value = f'<b>Matching products:</b> {len(filtered_options)} of {len(product_options)}'

# Connect category dropdowns to update function
bench_category_dropdown.observe(update_bench_product_list, names='value')
bench_line_dropdown.observe(update_bench_product_list, names='value')
bench_type_dropdown.observe(update_bench_product_list, names='value')
bench_class_dropdown.observe(update_bench_product_list, names='value')
bench_construction_dropdown.observe(update_bench_product_list, names='value')

# Baseline date range
baseline_start = widgets.DatePicker(
    description='Start:',
    value=min_date,
    disabled=False
)

baseline_end = widgets.DatePicker(
    description='End:',
    value=max_date,
    disabled=False
)

# Comparison date range
compare_start = widgets.DatePicker(
    description='Start:',
    value=min_date,
    disabled=False
)

compare_end = widgets.DatePicker(
    description='End:',
    value=max_date,
    disabled=False
)

benchmark_button = widgets.Button(description='Compare Ranges', button_style='primary')
benchmark_output = widgets.Output()

# Store dataframes for export
section6_exports = {}

def compare_benchmarks(b):
    """
    Compare weighted average prices between two date ranges using PER-PRODUCT
    baseline prices.
    
    Shows:
    1. Baseline Period Summary (per-product weighted averages)
    2. Comparison Period Summary
    3. Product-by-product comparison with Gain/Loss (color coded)
    4. Monthly breakdown vs baseline benchmark
    5. Export buttons for all tables
    """
    global section6_exports
    
    with benchmark_output:
        benchmark_output.clear_output(wait=True)
        
        # Check if Show All Rows is enabled
        show_all = bench_show_all_rows_toggle.value
        
        # Get date ranges from widgets
        b_start = pd.Timestamp(baseline_start.value)
        b_end = pd.Timestamp(baseline_end.value)
        c_start = pd.Timestamp(compare_start.value)
        c_end = pd.Timestamp(compare_end.value)
        
        # Get materials matching category filters
        matching_materials = get_materials_by_filters(
            category=bench_category_dropdown.value,
            line=bench_line_dropdown.value,
            mat_type=bench_type_dropdown.value,
            mat_class=bench_class_dropdown.value,
            construction=bench_construction_dropdown.value
        )
        
        # Filter baseline data
        baseline = df[(df['PO_Date'] >= b_start) & (df['PO_Date'] <= b_end)].copy()
        baseline = baseline[baseline['Material'].isin(matching_materials)]
        
        # Filter comparison data
        comparison = df[(df['PO_Date'] >= c_start) & (df['PO_Date'] <= c_end)].copy()
        comparison = comparison[comparison['Material'].isin(matching_materials)]
        
        # Filter by specific products if not 'ALL'
        selected_products = list(benchmark_product_filter.value)
        if 'ALL' not in selected_products:
            baseline = baseline[baseline['Material'].isin(selected_products)]
            comparison = comparison[comparison['Material'].isin(selected_products)]
        
        # =================================================================
        # HEADER - Show filters
        # =================================================================
        print("=" * 80)
        print("BENCHMARK COMPARISON")
        print("=" * 80)
        
        # Show active category filters
        active_filters = []
        if bench_category_dropdown.value != 'All':
            active_filters.append(f"Category: {bench_category_dropdown.value}")
        if bench_line_dropdown.value != 'All':
            active_filters.append(f"Line: {bench_line_dropdown.value}")
        if bench_type_dropdown.value != 'All':
            active_filters.append(f"Type: {bench_type_dropdown.value}")
        if bench_class_dropdown.value != 'All':
            active_filters.append(f"Class: {bench_class_dropdown.value}")
        if bench_construction_dropdown.value != 'All':
            active_filters.append(f"Construction: {bench_construction_dropdown.value}")
        
        if active_filters:
            print(f"Category Filters: {', '.join(active_filters)}")
        else:
            print("Category Filters: None (all categories)")
        if show_all:
            print("Display Mode: Showing ALL rows (scrollable)")
        print("=" * 80)
        
        # =================================================================
        # BASELINE PERIOD - PER PRODUCT WEIGHTED AVERAGES
        # =================================================================
        print("\n" + "=" * 80)
        print("BASELINE PERIOD - PER PRODUCT BENCHMARK PRICES")
        print("=" * 80)
        print(f"Date Range: {b_start.strftime('%Y-%m-%d')} to {b_end.strftime('%Y-%m-%d')}")
        
        if len(baseline) == 0:
            print("  NO DATA in baseline period!")
            return
        
        # Calculate per-product weighted averages for BASELINE
        baseline_products = baseline.groupby(['Material', 'Short Name']).agg({
            'Total_Value': 'sum',
            'Ordered_Qty': 'sum',
            'PO#': 'nunique'
        }).reset_index()
        baseline_products['Baseline_Price'] = (baseline_products['Total_Value'] / baseline_products['Ordered_Qty']).round(2)
        baseline_products['Baseline_Qty'] = baseline_products['Ordered_Qty']
        baseline_products['Baseline_POs'] = baseline_products['PO#']
        baseline_products['Baseline_Value'] = baseline_products['Total_Value'].round(2)
        
        print(f"  Products: {len(baseline_products)}")
        print(f"  Total POs: {baseline_products['Baseline_POs'].sum()}")
        print(f"  Total Quantity: {baseline_products['Baseline_Qty'].sum():,}")
        print(f"  Total FOB Value: ${baseline_products['Baseline_Value'].sum():,.2f}")
        print("=" * 80)
        
        # Show baseline per-product prices
        print("\nBaseline Prices by Product (your benchmark):\n")
        baseline_display = baseline_products[['Material', 'Short Name', 'Baseline_Price', 'Baseline_Qty', 'Baseline_POs']].copy()
        baseline_display = baseline_display.sort_values('Short Name')
        
        # Store for export
        section6_exports['baseline_prices'] = baseline_display.copy()
        
        baseline_display_fmt = baseline_display.copy()
        baseline_display_fmt['Baseline_Price'] = baseline_display_fmt['Baseline_Price'].apply(lambda x: f'${x:.2f}')
        baseline_display_fmt['Baseline_Qty'] = baseline_display_fmt['Baseline_Qty'].apply(lambda x: f'{x:,}')
        baseline_display_fmt.columns = ['Material', 'Product', 'Baseline Price', 'Baseline Qty', 'Baseline POs']
        
        # Display with scrollable option if Show All Rows is enabled
        if show_all:
            display(display_scrollable_table(baseline_display_fmt, max_height='400px', show_all=True))
        else:
            display(baseline_display_fmt)
        display(create_export_button(section6_exports['baseline_prices'], 'section6_baseline_prices', 'Export Baseline Prices to CSV'))
        
        # =================================================================
        # COMPARISON PERIOD SUMMARY
        # =================================================================
        print("\n" + "=" * 80)
        print("COMPARISON PERIOD")
        print("=" * 80)
        print(f"Date Range: {c_start.strftime('%Y-%m-%d')} to {c_end.strftime('%Y-%m-%d')}")
        
        if len(comparison) == 0:
            print("  NO DATA in comparison period!")
            return
        
        # Calculate per-product weighted averages for COMPARISON
        compare_products = comparison.groupby(['Material', 'Short Name']).agg({
            'Total_Value': 'sum',
            'Ordered_Qty': 'sum',
            'PO#': 'nunique'
        }).reset_index()
        compare_products['Compare_Price'] = (compare_products['Total_Value'] / compare_products['Ordered_Qty']).round(2)
        compare_products['Compare_Qty'] = compare_products['Ordered_Qty']
        compare_products['Compare_POs'] = compare_products['PO#']
        compare_products['Compare_Value'] = compare_products['Total_Value'].round(2)
        
        print(f"  Products: {len(compare_products)}")
        print(f"  Total POs: {compare_products['Compare_POs'].sum()}")
        print(f"  Total Quantity: {compare_products['Compare_Qty'].sum():,}")
        print(f"  Total FOB Value: ${compare_products['Compare_Value'].sum():,.2f}")
        print("=" * 80)
        
        # =================================================================
        # PER-PRODUCT COMPARISON WITH GAIN/LOSS
        # =================================================================
        print("\n" + "=" * 80)
        print("PER-PRODUCT COMPARISON: GAIN/LOSS ANALYSIS")
        print("=" * 80)
        print("Each product compared to ITS OWN baseline price (fair comparison)")
        print("Gain/Loss = (Baseline Price - Comparison Price) × Comparison Qty")
        print("  GREEN = GAIN (you paid LESS than baseline)")
        print("  RED = LOSS (you paid MORE than baseline)")
        print("")
        
        # Merge baseline and comparison (inner join - only products in BOTH periods)
        result = baseline_products[['Material', 'Short Name', 'Baseline_Price', 'Baseline_Qty', 'Baseline_POs']].merge(
            compare_products[['Material', 'Short Name', 'Compare_Price', 'Compare_Qty', 'Compare_POs']],
            on=['Material', 'Short Name'],
            how='inner'
        )
        
        # Calculate price change and gain/loss PER PRODUCT
        result['Price_Delta'] = (result['Compare_Price'] - result['Baseline_Price']).round(2)
        result['Price_Delta_Pct'] = ((result['Price_Delta'] / result['Baseline_Price']) * 100).round(2)
        
        # GAIN/LOSS: (Baseline - Comparison) × Comparison Qty
        # Positive = SAVINGS (paid less than baseline)
        # Negative = LOSS (paid more than baseline)
        result['Gain_Loss'] = ((result['Baseline_Price'] - result['Compare_Price']) * result['Compare_Qty']).round(2)
        
        # Sort by Gain/Loss (largest gains first)
        result = result.sort_values('Gain_Loss', ascending=False)
        
        # Store for export
        section6_exports['product_comparison'] = result.copy()
        
        # Count products by change direction
        products_gained = len(result[result['Gain_Loss'] > 0.01])
        products_lost = len(result[result['Gain_Loss'] < -0.01])
        products_neutral = len(result[(result['Gain_Loss'] >= -0.01) & (result['Gain_Loss'] <= 0.01)])
        
        print(f"Products in both periods: {len(result)}")
        print(f"  Products with SAVINGS: {products_gained}")
        print(f"  Products with LOSSES: {products_lost}")
        print(f"  Products NEUTRAL: {products_neutral}")
        print("")
        
        # Format for display with color coding
        display_df = result.copy()
        display_df['Baseline_Price'] = display_df['Baseline_Price'].apply(lambda x: f'${x:.2f}')
        display_df['Compare_Price'] = display_df['Compare_Price'].apply(lambda x: f'${x:.2f}')
        display_df['Baseline_Qty'] = display_df['Baseline_Qty'].apply(lambda x: f'{x:,}')
        display_df['Compare_Qty'] = display_df['Compare_Qty'].apply(lambda x: f'{x:,}')
        display_df['Price_Delta'] = display_df['Price_Delta'].apply(lambda x: color_currency(x, is_gain_loss=False, decimals=2))
        display_df['Price_Delta_Pct'] = display_df['Price_Delta_Pct'].apply(lambda x: color_pct(x, decimals=2))
        display_df['Gain_Loss'] = display_df['Gain_Loss'].apply(lambda x: color_currency(x, is_gain_loss=True, decimals=2))
        
        display_df.columns = ['Material', 'Product', 'Base Price', 'Base Qty', 'Base POs', 
                              'Comp Price', 'Comp Qty', 'Comp POs', 'Price Δ', 'Price Δ%', 'Gain/Loss']
        
        # Display with scrollable option if Show All Rows is enabled
        if show_all:
            html_table = display_df.to_html(escape=False, index=False)
            scrollable_html = f'<div style="max-height: 500px; overflow-y: auto; overflow-x: auto; border: 1px solid #ddd; padding: 5px;">{html_table}</div>'
            display(HTML(scrollable_html))
        else:
            display(HTML(display_df.to_html(escape=False, index=False)))
        display(create_export_button(section6_exports['product_comparison'], 'section6_product_comparison', 'Export Product Comparison to CSV'))
        
        # =================================================================
        # TOTAL GAIN/LOSS SUMMARY
        # =================================================================
        total_gain_loss = result['Gain_Loss'].sum()
        total_savings = result[result['Gain_Loss'] > 0]['Gain_Loss'].sum()
        total_losses = result[result['Gain_Loss'] < 0]['Gain_Loss'].sum()
        
        print("\n" + "=" * 80)
        print("TOTAL GAIN/LOSS SUMMARY (sum of all products)")
        print("=" * 80)
        
        if total_gain_loss >= 0:
            display(HTML(f'<h2 style="color: green;">NET RESULT: +${total_gain_loss:,.2f} (SAVINGS)</h2>'))
        else:
            display(HTML(f'<h2 style="color: red;">NET RESULT: -${abs(total_gain_loss):,.2f} (LOSS)</h2>'))
        
        display(HTML(f'<p style="color: green; font-size: 16px;">Total Savings: +${total_savings:,.2f}</p>'))
        display(HTML(f'<p style="color: red; font-size: 16px;">Total Losses: -${abs(total_losses):,.2f}</p>'))
        print("=" * 80)
        
        # =================================================================
        # DEVIATION BANDING CHARTS VS BASELINE
        # =================================================================
        # Histogram and Pie chart showing how comparison period purchases
        # deviate from baseline prices (similar to Section 5, but against baseline)
        # =================================================================
        print("\n\n" + "=" * 80)
        print("DEVIATION BANDING: COMPARISON VS BASELINE")
        print("=" * 80)
        print("How comparison period purchases deviate from baseline benchmark prices")
        print("")
        print(f"BASELINE Period: {b_start.strftime('%Y-%m-%d')} to {b_end.strftime('%Y-%m-%d')}")
        print(f"  - Products: {len(baseline_products)} | POs: {baseline_products['Baseline_POs'].sum()} | Qty: {baseline_products['Baseline_Qty'].sum():,}")
        print("")
        print(f"COMPARISON Period: {c_start.strftime('%Y-%m-%d')} to {c_end.strftime('%Y-%m-%d')}")
        print(f"  - Products: {len(compare_products)} | POs: {compare_products['Compare_POs'].sum()} | Qty: {compare_products['Compare_Qty'].sum():,}")
        print("")
        print("COLOR KEY: Green = Below Baseline (Savings) | Red = Above Baseline (Paid More)")
        print("=" * 80)
        
        # Calculate deviation for each comparison line item vs its product's baseline price
        comparison_with_baseline = comparison.merge(
            baseline_products[['Material', 'Baseline_Price']], 
            on='Material', 
            how='inner'
        )
        
        if len(comparison_with_baseline) > 0:
            comparison_with_baseline['Deviation'] = comparison_with_baseline['Purchase_Price'] - comparison_with_baseline['Baseline_Price']
            
            # Assign bands (same logic as Section 5)
            def assign_band_s6(deviation):
                if deviation <= -5:
                    return '$5+ below'
                elif deviation <= -4:
                    return '$4-5 below'
                elif deviation <= -3:
                    return '$3-4 below'
                elif deviation <= -2:
                    return '$2-3 below'
                elif deviation <= -1:
                    return '$1-2 below'
                elif deviation < 0:
                    return '$0-1 below'
                elif deviation < 1:
                    return '$0-1 above'
                elif deviation < 2:
                    return '$1-2 above'
                elif deviation < 3:
                    return '$2-3 above'
                elif deviation < 4:
                    return '$3-4 above'
                elif deviation < 5:
                    return '$4-5 above'
                else:
                    return '$5+ above'
            
            comparison_with_baseline['Band'] = comparison_with_baseline['Deviation'].apply(assign_band_s6)
            
            # Band order
            band_order_s6 = [
                '$5+ below', '$4-5 below', '$3-4 below', '$2-3 below',
                '$1-2 below', '$0-1 below', '$0-1 above', '$1-2 above',
                '$2-3 above', '$3-4 above', '$4-5 above', '$5+ above'
            ]
            
            # Sum total quantity per band
            band_qty_s6 = comparison_with_baseline.groupby('Band')['Ordered_Qty'].sum().reindex(band_order_s6, fill_value=0).reset_index()
            band_qty_s6.columns = ['Band', 'Total_Qty']
            
            # Also get PO count
            band_counts_s6 = comparison_with_baseline['Band'].value_counts().reindex(band_order_s6, fill_value=0).reset_index()
            band_counts_s6.columns = ['Band', 'PO_Count']
            
            band_summary_s6 = band_qty_s6.merge(band_counts_s6, on='Band')
            
            # ---------------------------------------------------------------------
            # HISTOGRAM
            # ---------------------------------------------------------------------
            colors_s6 = ['#2ca02c'] * 6 + ['#d62728'] * 6
            
            fig_hist_s6, ax_hist = plt.subplots(figsize=(12, 5))
            
            x_pos = range(len(band_order_s6))
            quantities = band_summary_s6['Total_Qty'].tolist()
            
            bars = ax_hist.bar(x_pos, quantities, color=colors_s6, edgecolor='black', linewidth=0.5)
            
            # Add value labels on top of bars
            for bar, qty in zip(bars, quantities):
                if qty > 0:
                    ax_hist.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(quantities)*0.01, 
                            f'{qty:,}', ha='center', va='bottom', fontsize=8, fontweight='bold')
            
            ax_hist.set_xticks(x_pos)
            ax_hist.set_xticklabels(band_order_s6, rotation=45, ha='right', fontsize=9)
            ax_hist.set_xlabel('Deviation from Baseline Price', fontsize=11)
            ax_hist.set_ylabel('Total Quantity (units)', fontsize=11)
            ax_hist.set_title('Comparison Period: Deviation from Baseline by TOTAL QUANTITY\n(Green = Below Baseline / Savings | Red = Above Baseline / Paid More)', 
                         fontsize=12, fontweight='bold')
            
            ax_hist.yaxis.grid(True, linestyle='--', alpha=0.7)
            ax_hist.set_axisbelow(True)
            
            plt.tight_layout()
            plt.show()
            
            # ---------------------------------------------------------------------
            # PIE CHART
            # ---------------------------------------------------------------------
            pie_colors_s6 = {
                '$5+ below': '#2ca02c', '$4-5 below': '#3cb371', '$3-4 below': '#32cd32',
                '$2-3 below': '#90ee90', '$1-2 below': '#98fb98', '$0-1 below': '#b2f2bb',
                '$0-1 above': '#ffb3b3', '$1-2 above': '#ff8080', '$2-3 above': '#ff6666',
                '$3-4 above': '#ff4d4d', '$4-5 above': '#e62e2e', '$5+ above': '#d62728'
            }
            
            # Filter out bands with zero quantity
            pie_data_s6 = band_summary_s6[band_summary_s6['Total_Qty'] > 0]
            
            if len(pie_data_s6) > 0:
                fig_pie_s6, ax_pie = plt.subplots(figsize=(10, 10))
                
                pie_labels = pie_data_s6['Band'].tolist()
                pie_values = pie_data_s6['Total_Qty'].tolist()
                pie_colors_list = [pie_colors_s6.get(b, '#888888') for b in pie_labels]
                
                # Calculate percentages
                total = sum(pie_values)
                percentages = [(v / total) * 100 for v in pie_values]
                
                explode = [0.03] * len(pie_values)
                
                def make_autopct_s6(values):
                    def my_autopct(pct):
                        if pct >= 3:
                            return f'{pct:.1f}%'
                        return ''
                    return my_autopct
                
                wedges, texts, autotexts = ax_pie.pie(
                    pie_values, 
                    labels=None,
                    colors=pie_colors_list,
                    autopct=make_autopct_s6(pie_values),
                    startangle=90,
                    explode=explode,
                    pctdistance=0.6,
                    wedgeprops={
                        'edgecolor': 'white',
                        'linewidth': 2.5,
                        'antialiased': True
                    },
                    shadow=True
                )
                
                for autotext in autotexts:
                    autotext.set_fontsize(11)
                    autotext.set_fontweight('bold')
                    autotext.set_color('black')
                
                legend_labels = [f'{label} ({pct:.1f}%)' for label, pct in zip(pie_labels, percentages)]
                
                ax_pie.legend(wedges, legend_labels, 
                          title="Deviation from Baseline",
                          loc='upper center', 
                          bbox_to_anchor=(0.5, -0.05),
                          ncol=3,
                          fontsize=9,
                          title_fontsize=10,
                          frameon=True,
                          fancybox=True,
                          shadow=True)
                
                ax_pie.set_title('Comparison Period: Quantity Distribution by Deviation from Baseline\n(Green = Savings | Red = Paid More)', 
                             fontsize=14, fontweight='bold')
                
                ax_pie.axis('equal')
                
                plt.tight_layout()
                plt.subplots_adjust(bottom=0.2)
                plt.show()
            
            # Summary statistics
            below_qty = comparison_with_baseline[comparison_with_baseline['Deviation'] < 0]['Ordered_Qty'].sum()
            above_qty = comparison_with_baseline[comparison_with_baseline['Deviation'] >= 0]['Ordered_Qty'].sum()
            total_qty = comparison_with_baseline['Ordered_Qty'].sum()
            
            print(f"\nDeviation Summary (vs Baseline):")
            print(f"  Quantity bought BELOW baseline: {below_qty:,} ({below_qty/total_qty*100:.1f}%) - GREEN = SAVINGS")
            print(f"  Quantity bought AT/ABOVE baseline: {above_qty:,} ({above_qty/total_qty*100:.1f}%) - RED = PAID MORE")
            
            # Band summary table
            print("\nBand Summary Table:")
            band_summary_display_s6 = band_summary_s6.copy()
            band_summary_display_s6['Qty %'] = (band_summary_display_s6['Total_Qty'] / band_summary_display_s6['Total_Qty'].sum() * 100).round(1).astype(str) + '%'
            band_summary_display_s6['Total_Qty'] = band_summary_display_s6['Total_Qty'].apply(lambda x: f'{x:,}')
            
            if show_all:
                display(display_scrollable_table(band_summary_display_s6, max_height='400px', show_all=True))
            else:
                display(band_summary_display_s6)
        
        # =================================================================
        # MONTHLY BREAKDOWN VS BASELINE
        # =================================================================
        print("\n\n" + "=" * 80)
        print("MONTHLY BREAKDOWN: GAIN/LOSS VS BASELINE")
        print("=" * 80)
        print("Each month's purchases compared to baseline price PER PRODUCT")
        print("")
        
        # Get baseline prices for joining
        baseline_prices = baseline_products[['Material', 'Short Name', 'Baseline_Price']].copy()
        
        # Calculate monthly data for comparison period
        monthly_comp = comparison.groupby(['Material', 'Short Name', 'PO_Month']).agg({
            'Total_Value': 'sum',
            'Ordered_Qty': 'sum'
        }).reset_index()
        monthly_comp['Monthly_Price'] = (monthly_comp['Total_Value'] / monthly_comp['Ordered_Qty']).round(2)
        
        # Merge with baseline prices
        monthly_comp = monthly_comp.merge(baseline_prices, on=['Material', 'Short Name'], how='inner')
        
        # Calculate gain/loss per month per product
        monthly_comp['Price_Delta'] = (monthly_comp['Monthly_Price'] - monthly_comp['Baseline_Price']).round(2)
        monthly_comp['Monthly_Gain_Loss'] = ((monthly_comp['Baseline_Price'] - monthly_comp['Monthly_Price']) * monthly_comp['Ordered_Qty']).round(2)
        
        # Create pivot for price delta
        pivot_delta = monthly_comp.pivot_table(
            index=['Material', 'Short Name'],
            columns='PO_Month',
            values='Price_Delta',
            aggfunc='first'
        ).round(2)
        
        # Store for export
        section6_exports['monthly_price_delta'] = pivot_delta.copy()
        
        print("Table 1: Price Delta vs Baseline (Comparison Price - Baseline Price)")
        print("GREEN = price went DOWN (savings), RED = price went UP (loss)\n")
        
        # Add baseline price column
        baseline_series = baseline_prices.set_index(['Material', 'Short Name'])['Baseline_Price']
        
        pivot_delta_display = pivot_delta.copy()
        for col in pivot_delta_display.columns:
            pivot_delta_display[col] = pivot_delta_display[col].apply(lambda x: color_currency(x, is_gain_loss=False, decimals=2))
        pivot_delta_display.insert(0, 'Baseline Price', baseline_series.apply(lambda x: f'${x:.2f}'))
        
        # Display with scrollable option if Show All Rows is enabled
        if show_all:
            html_table = pivot_delta_display.to_html(escape=False)
            scrollable_html = f'<div style="max-height: 400px; overflow-y: auto; overflow-x: auto; border: 1px solid #ddd; padding: 5px;">{html_table}</div>'
            display(HTML(scrollable_html))
        else:
            display(HTML(pivot_delta_display.to_html(escape=False)))
        display(create_export_button(section6_exports['monthly_price_delta'], 'section6_monthly_price_delta', 'Export Monthly Price Delta to CSV'))
        
        # Create pivot for gain/loss
        pivot_gainloss = monthly_comp.pivot_table(
            index=['Material', 'Short Name'],
            columns='PO_Month',
            values='Monthly_Gain_Loss',
            aggfunc='sum'
        ).round(2)
        
        # Store for export
        section6_exports['monthly_gain_loss'] = pivot_gainloss.copy()
        
        print("\n\nTable 2: Gain/Loss vs Baseline ($ amount)")
        print("GREEN = savings, RED = loss\n")
        
        pivot_gainloss_display = pivot_gainloss.copy()
        for col in pivot_gainloss_display.columns:
            pivot_gainloss_display[col] = pivot_gainloss_display[col].apply(lambda x: color_currency(x, is_gain_loss=True, decimals=2))
        
        # Add total column
        row_totals = pivot_gainloss.sum(axis=1).round(2)
        pivot_gainloss_display['TOTAL'] = row_totals.apply(lambda x: color_currency(x, is_gain_loss=True, decimals=2))
        
        # Display with scrollable option if Show All Rows is enabled
        if show_all:
            html_table = pivot_gainloss_display.to_html(escape=False)
            scrollable_html = f'<div style="max-height: 400px; overflow-y: auto; overflow-x: auto; border: 1px solid #ddd; padding: 5px;">{html_table}</div>'
            display(HTML(scrollable_html))
        else:
            display(HTML(pivot_gainloss_display.to_html(escape=False)))
        display(create_export_button(section6_exports['monthly_gain_loss'], 'section6_monthly_gain_loss', 'Export Monthly Gain/Loss to CSV'))
        
        # Monthly totals
        print("\n\nMonthly Gain/Loss Totals:")
        monthly_totals = pivot_gainloss.sum(axis=0)
        monthly_html = ""
        for month in monthly_totals.index:
            val = monthly_totals[month]
            if val >= 0:
                monthly_html += f'<div style="color: green; font-weight: bold; font-size: 14px;">{month}: +${val:,.2f} (savings)</div>'
            else:
                monthly_html += f'<div style="color: red; font-weight: bold; font-size: 14px;">{month}: -${abs(val):,.2f} (loss)</div>'
        display(HTML(monthly_html))
        
        monthly_grand_total = monthly_totals.sum()
        print("-" * 40)
        if monthly_grand_total >= 0:
            display(HTML(f'<h3 style="color: green;">MONTHLY GRAND TOTAL: +${monthly_grand_total:,.2f} (NET SAVINGS)</h3>'))
        else:
            display(HTML(f'<h3 style="color: red;">MONTHLY GRAND TOTAL: -${abs(monthly_grand_total):,.2f} (NET LOSS)</h3>'))

benchmark_button.on_click(compare_benchmarks)

# Display the interface
print("=" * 80)
print("BENCHMARK COMPARISON")
print("=" * 80)
print("Compare pricing between two date ranges (per-product comparison)")

print("\n1. FILTER BY CATEGORY (optional - narrow down products):")
display(widgets.VBox([
    widgets.HBox([bench_category_dropdown, bench_line_dropdown]),
    widgets.HBox([bench_type_dropdown, bench_class_dropdown]),
    bench_construction_dropdown
]))

print("\n2. SELECT PRODUCTS:")
display(bench_filter_status)
display(benchmark_product_filter)

print("\n3. SELECT DATE RANGES:")
print("   BASELINE Period (establishes benchmark price FOR EACH PRODUCT):")
display(widgets.HBox([baseline_start, baseline_end]))
print("   COMPARISON Period (compare each product to its baseline):")
display(widgets.HBox([compare_start, compare_end]))

print("\n4. DISPLAY OPTIONS:")
display(bench_show_all_rows_toggle)

print("\n5. COMPARE:")
display(benchmark_button)
display(benchmark_output)


---

## Quick Reference

| Section | Purpose |
|---------|----------|
| Section 2 | View monthly price trends per product |
| Section 3 | Calculate weighted average prices for date range |
| Section 4 | Analyze line item deviations from average |
| Section 5 | Visualize deviation distribution with banding |
| Section 6 | Compare prices between two periods |